<a id='intro'></a>

# Tools for exploring and calculating statistics over ICOS Atmospheric data


This notebook includes functions in Python for exploring ICOS data. The notebook is divided in the following parts:

- [Import Python  modules](#import_modules)
- [Data availability](#data_availability)
- [General-purpose functions](#general_funcs)
- [Bookeh help functions](#bookeh_help_funcs)
- [Map functions](#map_funcs)
- [Updating plot functions](#update_plot_funcs)
- [Plotting functions](#plotting_funcs)
- [Widget functions](#widget_funcs)
- [Control input](#control_input)
- [Statistics](#statistics)




Use the links to quickly navigate to the parts you are interested in. 

<a id='import_modules'></a>
<br>
<br>

## 1. Import modules

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;}

In [ ]:
#Import modules:
import numpy as np
from numpy import nan
import pandas as pd
from datetime import datetime
import requests
import fnmatch
from ipywidgets import interact, interact_manual, ColorPicker, Dropdown, SelectMultiple, Checkbox, DatePicker
from bokeh.io import show, reset_output, output_notebook

#Import ICOS tools:
from icoscp.sparql import sparqls
from icoscp.sparql.runsparql import RunSparql
from icoscp.station import station
from icoscp.cpb.dobj import Dobj

#Set the notebook as the selected output location:
reset_output()
output_notebook()

<a id='data_availability'></a>
<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>
<br>
<br>

## 2. Data availability
This part includes functions that return information about ICOS atmosphere stations that produce Level 1 and Level 2 CO, CO$_2$ and CH$_4$ data products.

In [ ]:
# Temporary fix to filter out non-icos atmospheric stations. This will be removed once the pylib is updated.
def get_icos_atmospheric_stations():
    query = \
    '''
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    prefix cpmeta: <http://meta.icos-cp.eu/ontologies/cpmeta/>
    prefix cpst: <http://meta.icos-cp.eu/ontologies/stationentry/>
    select ?uri ?id ?name ?icosClass ?country ?siteType ?lat ?lon ?elevation ?stationTheme ?firstName ?lastName ?email
    from <http://meta.icos-cp.eu/resources/icos/>
    from <http://meta.icos-cp.eu/resources/extrastations/>
    from <http://meta.icos-cp.eu/resources/stationentry/>
    where {
        {
            select ?uri ?id (sample(?pers) as ?piOpt)  where{
                ?uri cpmeta:hasStationId ?id .
                OPTIONAL{
                    ?memb cpmeta:atOrganization ?uri ; cpmeta:hasRole <http://meta.icos-cp.eu/resources/roles/PI> .
                    filter not exists {?memb cpmeta:hasEndTime []}
                    ?pers cpmeta:hasMembership ?memb
                }
            }
            group by ?uri ?id
        }
        ?uri a ?stationTheme .
        bind(coalesce(?piOpt, <http://dummy>) as ?pi)
        OPTIONAL{
            ?pi cpmeta:hasFirstName ?firstName .
            ?pi cpmeta:hasLastName ?lastName
        }
        OPTIONAL{?pi cpmeta:hasEmail ?email}
        OPTIONAL {
            {
                ?provSt cpst:hasProductionCounterpart ?prodUriStr .
                filter(iri(?prodUriStr) = ?uri)
            }
            ?provSt cpst:hasSiteType ?siteType .
        }
        OPTIONAL {?uri cpmeta:hasStationClass ?icosClass}
        OPTIONAL {?uri cpmeta:hasName ?name  }
        OPTIONAL {?uri cpmeta:countryCode ?country }
        OPTIONAL {?uri cpmeta:hasLatitude ?lat }
        OPTIONAL {?uri cpmeta:hasLongitude ?lon }
        OPTIONAL {?uri cpmeta:hasElevation ?elevation }
    }
    '''
    # Fetch all stations using the query above.
    all_stations = RunSparql(sparql_query=query, output_format='pandas').run().transpose()
    icos_atm_stations = list()
    for index in all_stations:
        station_info = all_stations[index]
        # Only stations with the correct theme and icos class will be included.
        if station_info.stationTheme == 'http://meta.icos-cp.eu/ontologies/cpmeta/AS' and station_info.icosClass is not None:
            icos_atm_stations.append(station.get(station_info.id))
    return icos_atm_stations
# End of temporary fix.

def create_lookup_df_atc_L1(station_obj_ls):
    
    """
    Project:     'ICOS Carbon Portal'
    Created:      Wed Mar 28 17:40:00 2019
    Last Changed: Tue Nov 17 14:00:00 2020
    Version:      0.1.3
    Author(s):    Karolina Pantazatou
    
    Description:  Return a pandas dataframe with information for all available ICOS
                  Level-1 Atmospheric Data Products.
    
    Input:        1. List of ICOS atmospheric station objects
                    (var_name: 'station_obj_ls', var_type: List)
    
    Output:       pandas dataframe
                  columns:
                      1. URL to ICOS station Landing Page
                         (var_name: 'station', var_type: String)
                      2. instrument sampling height expressed in m a.g.l.
                         (var_name: 'samplingheight', var_type: String)
                      3. URL to ICOS RI Data Object Landing Page
                         (var_name: 'dobj', var_type: String)
                      4. Sampling start time
                         (var_name: 'timeStart', var_type: String)
                      5. Sampling end time
                         (var_name: 'timeEnd', var_type: String)
                      6. Data product
                         (var_name: 'specLabel', var_type: String)
                      7. Station name - long
                         (var_type: 'stationName', var_type: String)
                      8. Country in which the station is located
                         (var_type: 'country', var_type: String)
                      9. Station latitude
                         (var_type: 'lat', var_type: Float)
                     10. Station longitude
                         (var_type: 'lon', var_type: Float)
                     11. Station PI first name
                         (var_type: 'firstName', var_type: String)
                     12. Station PI last name
                         (var_type: 'lastName', var_type: String)
                     13. PI contact info
                         (var_type: 'email', var_type: String)
                     14. 3-character Station ID
                         (var_name: 'stationId', var_type: String)
    """
    
    #Function that adds a column with the ICOS station long name:
    def add_st_name_col(st_info_ls, st_col_ls, lookup_df):
        
        #Check that input parameters are lists:
        if((isinstance(st_info_ls, list)) & (isinstance(st_col_ls, list)) & (len(st_info_ls)==len(st_col_ls))):
            
            #Loop through items in station info list:
            for i in range(len(st_info_ls)):

                #Add column with current station info item:
                lookup_df[st_col_ls[i]] = lookup_df.apply (lambda row: st_info_ls[i], axis=1)

            #Return dataframe:
            return lookup_df
        
        #If input parameter is not a list:
        else:
            
            print('Error!\nInput parameters st_info_ls & st_col_ls have to be lists of same size.')
            
            #Return empty dataframe:
            return pd.DataFrame()
  
    
    #Create a pandas dataframe with info about ICOS L1 AS stations and CO2, CO & CH4 data products:
    lookup_atc_L1_df  = pd.concat([add_st_name_col([st.name, st.country, st.lat, st.lon,
                                                    st.firstName, st.lastName, st.email],
                                                   ['stationName', 'country', 'lat', 'lon',
                                                    'firstName', 'lastName', 'email'],
                                                   st.data()[['station',
                                                              'samplingheight',
                                                              'dobj',
                                                              'timeStart',
                                                              'timeEnd',
                                                              'specLabel']].loc[(st.data().specLabel=='ICOS ATC NRT CO2 growing time series') |
                                                                                (st.data().specLabel=='ICOS ATC NRT CH4 growing time series') |
                                                                                (st.data().specLabel=='ICOS ATC NRT CO growing time series')])
                                                   for st in station_obj_ls]).reset_index().drop('index', axis=1)

    #Add column with stationId (last 3 chars in station landing page URL):
    lookup_atc_L1_df['stationId'] = [st_str[-3:] for st_str in lookup_atc_L1_df.station]
    
    #Return dataframe:
    return lookup_atc_L1_df

In [ ]:
def create_lookup_df_atc_L2(station_obj_ls):
    
    """
    Project:     'ICOS Carbon Portal'
    Created:      Wed Mar 28 17:40:00 2019
    Last Changed: Tue Nov 17 14:00:00 2020
    Version:      0.1.3
    Author(s):    Karolina Pantazatou
    
    Description:  Return a pandas dataframe with information for all available ICOS
                  Level-2 Atmospheric Data Products.
    
    Input:        1. List of ICOS atmospheric station objects (var_name: 'station_obj_ls', var_type: List)
    
    Output:       pandas dataframe
                  columns:
                      1. URL to ICOS station Landing Page
                         (var_name: 'station', var_type: String)
                      2. instrument sampling height expressed in m a.g.l.
                         (var_name: 'samplingheight', var_type: String)
                      3. URL to ICOS RI Data Object Landing Page
                         (var_name: 'dobj', var_type: String)
                      4. Sampling start time
                         (var_name: 'timeStart', var_type: String)
                      5. Sampling end time
                         (var_name: 'timeEnd', var_type: String)
                      6. Data product
                         (var_name: 'specLabel', var_type: String)
                      7. Station name - long
                         (var_type: 'stationName', var_type: String)
                      8. Country in which the station is located
                         (var_type: 'country', var_type: String)
                      9. Station latitude
                         (var_type: 'lat', var_type: Float)
                     10. Station longitude
                         (var_type: 'lon', var_type: Float)
                     11. Station PI first name
                         (var_type: 'firstName', var_type: String)
                     12. Station PI last name
                         (var_type: 'lastName', var_type: String)
                     13. PI contact info
                         (var_type: 'email', var_type: String)
                     14. 3-character Station ID
                         (var_name: 'stationId', var_type: String)
    """
    
    #Function that adds a column with the ICOS station long name:
    def add_st_name_col(st_info_ls, st_col_ls, lookup_df):
        
        #Check that input parameters are lists:
        if((isinstance(st_info_ls, list)) & (isinstance(st_col_ls, list)) & (len(st_info_ls)==len(st_col_ls))):
            
            #Loop through items in station info list:
            for i in range(len(st_info_ls)):

                #Add column with current station info item:
                lookup_df[st_col_ls[i]] = lookup_df.apply (lambda row: st_info_ls[i], axis=1)

            #Return dataframe:
            return lookup_df
        
        #If input parameter is not a list:
        else:
            
            print('Error!\nInput parameters st_info_ls & st_col_ls have to be lists of same size.')
            
            #Return empty dataframe:
            return pd.DataFrame()
        
        
    #Create a pandas dataframe with info about ICOS L2 AS stations and CO2, CO & CH4 data products:
    lookup_atc_L2_df  = pd.concat([add_st_name_col([st.name, st.country, st.lat, st.lon,
                                                    st.firstName, st.lastName, st.email],
                                                   ['stationName', 'country', 'lat', 'lon',
                                                    'firstName', 'lastName', 'email'],
                                                   st.data()[['station',
                                                              'samplingheight',
                                                              'dobj',
                                                              'timeStart',
                                                              'timeEnd',
                                                              'specLabel']].loc[(st.data().specLabel=='ICOS ATC CO2 Release') |
                                                                                (st.data().specLabel=='ICOS ATC CH4 Release') |
                                                                                (st.data().specLabel=='ICOS ATC CO Release')])
                                   for st in station_obj_ls]).reset_index().drop('index', axis=1)

    #Add column with stationId (last 3 chars in station landing page URL):
    lookup_atc_L2_df['stationId'] = [st_str[-3:] for st_str in lookup_atc_L2_df.station]
    
    #Return dataframe:
    return lookup_atc_L2_df

<a id='general_funcs'></a>
<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>
<br>
<br>

## 3. General-purpose functions


In [ ]:
def printmd(string):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri May 10 12:00:00 2019
    Last Changed:     Fri May 10 12:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that allows you to print the string input parameter using
                      markdown formatting code.
                      
    Input parameters: String of characters
                      (var_name: 'string', var_type: String)

    Output:           String     
    
    """
    
    #import module:
    from IPython.display import Markdown, display
    
    #Enable displaying string with markdown code:
    display(Markdown(string))

In [ ]:
def get_country_fullname_from_iso3166_2char(countryCode):
    
    #Get iso 3166 translation pandas dataframe:
    country_names_codes_iso3166 = pd.read_csv('data/country_names_codes_iso_3166.csv',
                                              header=0,
                                              delimiter=';')
    
    #Check if the input is a valid 2-character long ISO 3166 country code:
    if ('SE' in country_names_codes_iso3166.Alpha_2_code.values):
        
        #Return the fullname of a country based on the given iso 3166 2-character country code:
        return country_names_codes_iso3166.Country.loc[country_names_codes_iso3166.Alpha_2_code==countryCode].values[0]

    
    #If the input is not a valid 2-character long ISO 3166 country code:
    else:
        print('Error! Invalid ISO 3166 2-char country code')
    

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='bookeh_help_funcs'></a>

<br>
<br>

## 4. Bokeh help functions
This part includes help functions for displaying Bokeh plots. The help functions handle:
- visualization of secondary y-axis in Bokeh plots (aligning 2nd-yaxis to primary yaxis)
- visualization of secondary & 3rd y-axis in Bokeh plots (aligning 2nd-yaxis and 3rd-yaxis to primary yaxis)
- automatic assignment of Bokeh colormaps to plots based on number of information layers

In [ ]:
def rounddown_100(x):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 09:00:00 2018
    Last Changed:     Tue May 07 09:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes a number as input and floors it to the nearest "100".
                      
    Input parameters: Number (var_name: 'x', var_type: Integer or Float)

    Output:           Float
    
    """
    
    #Import module:
    import numbers
    
    #Check if input parameter is numeric:
    if(isinstance(x, numbers.Number)==True):

        #If the number is an integral multiple of 100:
        if(((x/100.0)%2==0) or (x<=0) or (x==100)):

            return(int(x / 100.0) * 100) - 100
        
        #If the input number is NOT an integral multiple of 100:
        else:

            return(int(x / 100.0) * 100)

    #If input parameter is not numeric, prompt an error message:
    else:
        print("Input parameter is not numeric!")
    
    

In [ ]:
def roundup_100(x):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 09:00:00 2018
    Last Changed:     Tue May 07 09:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes a number as input and rounds it up to the nearest "100".
                      
    Input parameters: Number (var_name: 'x', var_type: Integer or Float)

    Output:           Float
    
    """
    
    #Import modules:
    import math
    import numbers
    
    #Check if input parameter is numeric:
    if(isinstance(x, numbers.Number)==True):
        
        #for integral mulitples of 100 and for the special cases of 100, 0 and -100:
        if(((x/100.0)%2==0) or (x==100) or (x==-100)):  
            return int(math.ceil(x / 100.0)) * 100 + 100

        else:
            return int(math.ceil(x / 100.0)) * 100
    
    #If input parameter is not numeric, prompt an error message:
    else:
        print("Input parameter is not numeric!")
    

In [ ]:
def rounddown_20(x):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 09:00:00 2018
    Last Changed:     Tue May 07 09:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes a number as input and floors it to the nearest "20".
                      
    Input parameters: Number (var_name: 'x', var_type: Integer or Float)

    Output:           Float
    
    """
    
    #Import module:
    import math
    import numbers
    
    #Check if input parameter is numeric:
    if(isinstance(x, numbers.Number)==True):
    
        #If the 2nd digit from the decimal point is an even number:
        if(int(x/10.0)%2==0):

            return(int(x / 10.0) * 10) - 20

        #If the 2nd digit from the decimal point is an odd number:
        else:

            return(int(x / 10.0) * 10) - 10
        
    #If input parameter is not numeric, prompt an error message:
    else:
        print("Input parameter is not numeric!")

In [ ]:
def roundup_20(x):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 09:00:00 2018
    Last Changed:     Tue May 07 09:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes a number as input and rounds it up to the closest "20".
                      
    Input parameters: Number (var_name: 'x', var_type: Integer or Float)

    Output:           Float
    
    """
    
    #Import module:
    import math
    import numbers
    
    #Check if input parameter is numeric:
    if(isinstance(x, numbers.Number)==True):
    
        #for positive numbers, multiples of 20.0:
        if((x>=0)&(((x/10.0)%20)%2 == 0)):  
            return int(math.ceil(x / 10.0)) * 10 +20

        #for positive numbers with an even number as 2nd digit:
        elif((x>0)&(int(x/10.0)%2==0)):
            return int(math.ceil(x / 10.0)) * 10 +10

        #for positive and negative numbers, whose 2nd digit is an odd number (except for i in [-1,-9]):
        elif(int(x/10.0)%2!=0):
            return int((x / 10.0)) * 10 +10

        #for negative numbers, whose 1st or 2nd digit is an even number:
        elif((x<-10) & (int(x)%2==0)):   
            return int((x / 10.0)) * 10 +20

        else:
            return 0
    
    #If input parameter is NOT numeric, prompt an error message:
    else:
        print("Input parameter is not numeric!")
    

In [ ]:
def set_yranges_2y(y1_min, y1_max, y2_min, y2_max, y1_step, y2_step ,new_yrange_name):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes the primary and secondary y-axis min/max values as well as
                      the step values for every y-axis and the secondary y-axis new range name as input
                      parameters, performs computations so that the two axes are alligned and returns
                      their corresponding RangeId objects. Works only for Bokeh plots.
                      
    Input parameters: 1. Min value of primary y-axis (var_name: 'y1_min', var_type: Integer or Float)
                      2. Max value of primary y-axis (var_name: 'y1_max', var_type: Integer or Float)
                      3. Min value of secondary y-axis (var_name: 'y2_min', var_type: Integer or Float)
                      4. Max value of secondary y-axis (var_name: 'y2_max', var_type: Integer or Float)
                      5. Step of primary y-axis (var_name: 'y1_step', var_type: Integer or Float)
                      6. Step of secondary y-axis (var_name: 'y2_step', var_type: Integer or Float)
                      7. Name of new yrange object for secondary y-axis
                         (var_name: "new_yrange_name", var_type: Bokeh Plot yrange object)

    Output:           Bokeh Plot yrange objects for primary and secondary y-axes.
    
    """
    
    #import modules:
    import numpy as np
    from bokeh.models import Range1d
          
    #yrange and tick function for plot with primary and secondary y-axis:
    yticks1 = np.arange(y1_min, y1_max + y1_step, y1_step)
    yticks2 = np.arange(y2_min, y2_max + y2_step, y2_step)

    #Get difference in total number of ticks between primary and secondary y-axis:  
    diff = abs(len(yticks2)-len(yticks1))

    #Get how many times the step needs to be added to start and end:
    num_of_steps = int(diff/2)
    
    #If the primary and the secondary y-axis have the same number of ticks:
    if(diff==0):

        #Set the range of the 1st y-axis:
        y_range = Range1d(start=y1_min, end=y1_max)

        #Set the 2nd y-axis, range-name, range:
        extra_y_ranges = {new_yrange_name: Range1d(start=y2_min, end=y2_max)}

        
    #If the primary y-axis has fewer ticks than the secondary y-axis:    
    elif(len(yticks2)>len(yticks1)):
    
        #If the difference in ticks between the two axes is an odd number:
        if(diff%2==1):

            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min-(y1_step*(num_of_steps+1)), end=y1_max+(y1_step*num_of_steps))

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges = {new_yrange_name: Range1d(start=y2_min, end=y2_max)}

        #If the difference in ticks between the two axes is an even number:
        else:
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min-(y1_step*num_of_steps), end=y1_max+(y1_step*num_of_steps))

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges = {new_yrange_name: Range1d(start=y2_min, end=y2_max)}
            
            
    #If the primary y-axis has more ticks than the secondary y-axis, e.g. len(yticks1)>len(yticks2_test):
    else:
        
        #If the difference in ticks between the two axes is an odd number:
        if(diff%2==1):
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min, end=y1_max)

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges = {new_yrange_name: Range1d(start=y2_min - (y2_step*(num_of_steps)), end=y2_max + (y2_step*(num_of_steps+1)))}

        #If the difference in ticks between the two axes is an even number:
        else:
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min, end=y1_max)

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges = {new_yrange_name: Range1d(start=y2_min - (y2_step*num_of_steps), end=y2_max + (y2_step*num_of_steps))}
              
        
    #Return y-range for primary and secondary y-axes:
    return y_range, extra_y_ranges


In [ ]:
def set_yranges_3y(y1_min, y1_max, y2_min, y2_max, y3_min, y3_max, y1_step, y2_step, y3_step):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes the primary, secondary and third y-axis min/max values as well as
                      the step values for every y-axis as input parameters, performs computations so that the
                      three axes are alligned and returns their corresponding RangeId objects.
                      Works only for Bokeh plots.
                      
    Input parameters: 1. Min value of primary y-axis (var_name: 'y1_min', var_type: Integer or Float)
                      2. Max value of primary y-axis (var_name: 'y1_max', var_type: Integer or Float)
                      3. Min value of secondary y-axis (var_name: 'y2_min', var_type: Integer or Float)
                      4. Max value of secondary y-axis (var_name: 'y2_max', var_type: Integer or Float)
                      5. Min value of third y-axis (var_name: 'y3_min', var_type: Integer or Float)
                      6. Max value of third y-axis (var_name: 'y3_max', var_type: Integer or Float)
                      7. Step of primary y-axis (var_name: 'y1_step', var_type: Integer or Float)
                      8. Step of secondary y-axis (var_name: 'y2_step', var_type: Integer or Float)
                      9. Step of third y-axis (var_name: 'y3_step', var_type: Integer or Float)

    Output:           Bokeh Plot yrange objects for primary and secondary y-axes.
    
    """
    
    #import modules:
    import numpy as np
    from bokeh.models import Range1d
          
    #yrange and tick function for plot with primary and secondary y-axis:
    yticks1 = np.arange(y1_min, y1_max + y1_step, y1_step)
    yticks2 = np.arange(y2_min, y2_max + y2_step, y2_step)
    yticks3 = np.arange(y3_min, y3_max + y3_step, y3_step)
    
    #Get the number of ticks per y-axis:
    y1_num_of_ticks = len(yticks1)
    y2_num_of_ticks = len(yticks2)
    y3_num_of_ticks = len(yticks3)

    #Get difference in total number of ticks between primary and secondary y-axis:  
    diff_12 = abs(len(yticks2)-len(yticks1))
    diff_13 = abs(len(yticks3)-len(yticks1))
    diff_23 = abs(len(yticks3)-len(yticks2))

    #Get how many times the step needs to be added to start and end:
    num_of_steps_12 = int(diff_12/2)
    num_of_steps_13 = int(diff_13/2)
    num_of_steps_23 = int(diff_23/2)
    
    
    #If the primary, secondary and 3rd y-axis have the same number of ticks:
    if((diff_12==0) and (diff_13==0) and (diff_23==0)):

        #Set the range of the 1st y-axis:
        y_range = Range1d(start=y1_min, end=y1_max)

        #Set the 2nd y-axis, range-name, range:
        extra_y_ranges_1 = Range1d(start=y2_min, end=y2_max)
        
        #Set the 3rd y-axis, range-name, range:
        extra_y_ranges_2 = Range1d(start=y3_min, end=y3_max)

        
        
    #if y-axis 1 is the axis with the highest number of ticks:
    elif(max(y1_num_of_ticks, y2_num_of_ticks, y3_num_of_ticks)==y1_num_of_ticks):
        
        #Check if the difference between y-axis 1 and the other axes is an even number:
        if((diff_12%2==0) and (diff_13%2==0)):
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min, end=y1_max)

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges_1 = Range1d(start=y2_min - (y2_step*num_of_steps_12),
                                       end=y2_max + (y2_step*num_of_steps_12))
            
            #Set the 3rd y-axis, range-name, range:
            extra_y_ranges_2 = Range1d(start=y3_min - (y3_step*num_of_steps_13), 
                                       end=y3_max + (y3_step*num_of_steps_13))

        
        #Check if the difference between y-axis 1 and the other axes is an odd number:
        elif((diff_12%2==1) and (diff_13%2==1)):
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min, end=y1_max)

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges_1 = Range1d(start=y2_min - (y2_step*(num_of_steps_12)),
                                       end=y2_max + (y2_step*(num_of_steps_12+1)))
            
            #Set the 3rd y-axis, range-name, range:
            extra_y_ranges_2 = Range1d(start=y3_min - (y3_step*(num_of_steps_13)),
                                       end=y3_max + (y3_step*(num_of_steps_13+1)))

            
        #Check if the difference between y-axis 1 and the other axes is an even/odd number:
        elif((diff_12%2==0) and (diff_13%2==1)):
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min, end=y1_max)

            #Set the 2nd y-axis, range-name, range: --- > even diff
            extra_y_ranges_1 = Range1d(start=y2_min - (y2_step*num_of_steps_12),
                                       end=y2_max + (y2_step*num_of_steps_12))
            
            #Set the 3rd y-axis, range-name, range: --- > odd diff
            extra_y_ranges_2 = Range1d(start=y3_min - (y3_step*(num_of_steps_13)),
                                       end=y3_max + (y3_step*(num_of_steps_13+1)))
         
        
        #Check if the difference between y-axis 1 and the other axes is an odd/even number:
        #I.e. (diff_12%2==1) and (diff_13%2==0)
        else:
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min, end=y1_max)

            #Set the 2nd y-axis, range-name, range: --- > odd diff
            extra_y_ranges_1 = Range1d(start=y2_min - (y2_step*(num_of_steps_12)),
                                       end=y2_max + (y2_step*(num_of_steps_12+1)))
            
            #Set the 3rd y-axis, range-name, range: --- > even diff
            extra_y_ranges_2 = Range1d(start=y3_min - (y3_step*num_of_steps_13), 
                                       end=y3_max + (y3_step*num_of_steps_13))
        
      
    
    #if y-axis 2 is the axis with the highest number of ticks:
    elif(max(y1_num_of_ticks, y2_num_of_ticks, y3_num_of_ticks)==y2_num_of_ticks):
        
        #Check if the difference between y-axis 2 and the other axes is an even number:
        if((diff_12%2==0) and (diff_23%2==0)):
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min-(y1_step*num_of_steps_12),
                              end=y1_max+(y1_step*num_of_steps_12))

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges_1 = Range1d(start=y2_min, end=y2_max)
            
            #Set the 3rd y-axis, range-name, range:
            extra_y_ranges_2 = Range1d(start=y3_min-(y3_step*num_of_steps_23),
                                       end=y3_max+(y3_step*num_of_steps_23))
            
            
        #Check if the difference between y-axis 2 and the other axes is an odd number:
        elif((diff_12%2==1) and (diff_23%2==1)):
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min-(y1_step*(num_of_steps_12+1)),
                              end=y1_max+(y1_step*num_of_steps_12))

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges_1 = Range1d(start=y2_min, end=y2_max)
            
            #Set the 3rd y-axis, range-name, range:
            extra_y_ranges_2 = Range1d(start=y3_min-(y3_step*(num_of_steps_23+1)),
                                       end=y3_max+(y3_step*num_of_steps_23))
      
            
        #Check if the difference between y-axis 2 and the other axes is an even/odd number:
        elif((diff_12%2==0) and (diff_23%2==1)):
            
            #Set the range of the 1st y-axis: --- > even diff
            y_range = Range1d(start=y1_min-(y1_step*num_of_steps_12),
                              end=y1_max+(y1_step*num_of_steps_12))

            #Set the 2nd y-axis, range-name, range: 
            extra_y_ranges_1 = Range1d(start=y2_min, end=y2_max)
            
            #Set the 3rd y-axis, range-name, range: --- > odd diff
            extra_y_ranges_2 = Range1d(start=y3_min-(y3_step*(num_of_steps_23+1)),
                                       end=y3_max+(y3_step*num_of_steps_23))
            
         
        #Check if the difference between y-axis 2 and the other axes is an odd/even number:
        #I.e. (diff_12%2==1) and (diff_23%2==0)
        else:
            
            #Set the range of the 1st y-axis: --- > odd diff
            y_range = Range1d(start=y1_min-(y1_step*(num_of_steps_12+1)),
                              end=y1_max+(y1_step*num_of_steps_12))

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges_1 = Range1d(start=y2_min, end=y2_max)
            
            #Set the 3rd y-axis, range-name, range: --- > even diff
            extra_y_ranges_2 = Range1d(start=y3_min-(y3_step*num_of_steps_23),
                                       end=y3_max+(y3_step*num_of_steps_23))
            
    
    
    #if y-axis 3 is the axis with the highest number of ticks:
    elif(max(y1_num_of_ticks, y2_num_of_ticks, y3_num_of_ticks)==y3_num_of_ticks):
        
        #Check if the difference between y-axis 3 and the other axes is an even number:
        if((diff_13%2==0) and (diff_23%2==0)):
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min-(y1_step*num_of_steps_13),
                              end=y1_max+(y1_step*num_of_steps_13))

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges_1 = Range1d(start=y2_min-(y2_step*num_of_steps_23),
                                       end=y2_max+(y2_step*num_of_steps_23))
            
            #Set the 3rd y-axis, range-name, range:
            extra_y_ranges_2 = Range1d(start=y3_min, end=y3_max)
            
        
        #Check if the difference between y-axis 3 and the other axes is an odd number:
        elif((diff_13%2==1) and (diff_23%2==1)):
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min-(y1_step*(num_of_steps_13+1)),
                              end=y1_max+(y1_step*num_of_steps_13))
            
            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges_1 = Range1d(start=y2_min-(y2_step*(num_of_steps_23+1)),
                                       end=y2_max+(y2_step*num_of_steps_23))
            
            #Set the 3rd y-axis, range-name, range:
            extra_y_ranges_2 = Range1d(start=y3_min, end=y3_max)
            
            
            
        #Check if the difference between y-axis 3 and the other axes is an even/odd number:
        elif((diff_13%2==0) and (diff_23%2==1)):
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min-(y1_step*num_of_steps_13),
                              end=y1_max+(y1_step*num_of_steps_13))
            
            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges_1 = Range1d(start=y2_min-(y2_step*(num_of_steps_23+1)),
                                       end=y2_max+(y2_step*num_of_steps_23))
            
            #Set the 3rd y-axis, range-name, range:
            extra_y_ranges_2 = Range1d(start=y3_min, end=y3_max)
            
            
        #Check if the difference between y-axis 3 and the other axes is an odd/even number:
        #I.e. (diff_13%2==1) and (diff_23%2==0)
        else:
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min-(y1_step*(num_of_steps_13+1)),
                              end=y1_max+(y1_step*num_of_steps_13))
            
            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges_1 = Range1d(start=y2_min-(y2_step*num_of_steps_23),
                                       end=y2_max+(y2_step*num_of_steps_23))
            
            #Set the 3rd y-axis, range-name, range:
            extra_y_ranges_2 = Range1d(start=y3_min, end=y3_max)
        
    else:
        y_range = None
        extra_y_ranges_1 = None
        extra_y_ranges_2 = None
      
        
    #Return y-range for primary and secondary y-axes:
    return y_range, extra_y_ranges_1, extra_y_ranges_2


In [ ]:
def get_colormap(num_of_items):
      
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri Apr 04 17:00:00 2019
    Last Changed:     Fri Apr 04 17:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes an integer representing the total number of items that should receive
                      a sepparate color and  returns a colormap (i.e. a list of strings, where every string
                      represents a different color in hexadecimal code) with the same amount of colors.
                      The function can return colormaps for 1 - 256 items.
                      
    Input parameters: Number of items to be colored in a sepparate color
                      (var_name: 'num_of_items', var_type: Integer)

    Output:           List of strings (colormap)
    
    """
    
    #Check input:
    if(isinstance(num_of_items, int)):
    
        #import module:
        from bokeh.palettes import all_palettes, Colorblind, magma

        #Check the number of items to be colored (1-2 items):
        if((num_of_items>0) and (num_of_items<3)):
            return ['#2b83ba','#fdae61'] #return colormap selection
        
        #Check the number of items to be colored (3-8 items):
        elif((num_of_items>2) and (num_of_items<9)):
            return all_palettes['Colorblind'][num_of_items] #return colormap selection
        
        #Check the number of items to be colored (9-256 items):
        elif((num_of_items>8) and (num_of_items<257)):
            return magma(num_of_items) #return colormap selection         
        
        #If the number of items exceeds the number 256:
        else:
            print('Error! Number of items to be colored is zero or higher than 256.')
    
    #If the input parameter is not an integer:
    else:
        print('Input is not an integer.')
        

<a id='map_funcs'></a>
<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>
<br>
<br>

## 5. Map functions
This part includes functions that åroduce interactive maps with folium.

In [ ]:
def plotmap(stations_df, selected_station, basemap, d_icon='cloud', icon_col='orange'):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue Feb 04 10:40:00 2020
    Last Changed:     Wed Nov 18 15:00:00 2020
    Version:          0.1.2
    Author(s):        Karolina Pantazatou
    
    Description:      Function that takes a dataframe containing info about ICOS Stations and the 3-character
                      station code of a selected station as input and returns an interactive Folium Map, with
                      the location of the selected station highlighted in red. 
                      Folium (URL): https://python-visualization.github.io/folium/quickstart.html
                      
    Input parameters: 1. Dataframe with Information regarding ICOS Stations
                         (var_name: 'stations_df', var_type: Pandas Dataframe)
                      2. Station 3-character Code
                         (var_name: 'selected_station', var_type: String)

    Output:           Folium Map (Folium Map Object)
    
    """
    
    #Import modules:
    import folium

    #Check what type of basemap is selected:
    if(basemap=='Imagery'):
        
        #Create folium map-object:
        m = folium.Map(location=[float(stations_df.loc[stations_df.stationId==selected_station].lat.values[0]),
                                 float(stations_df.loc[stations_df.stationId==selected_station].lon.values[0])],
                       zoom_start=5,
                       tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                       attr = 'Esri',
                       name = 'Esri Satellite',
                       overlay = False,
                       control = True)
        
    else:
        
        #Create folium map-object:
        m = folium.Map(
            location=[float(stations_df.loc[stations_df.stationId==selected_station].lat.values[0]),
                      float(stations_df.loc[stations_df.stationId==selected_station].lon.values[0])],
            zoom_start=4)

    #Add marker-tooltip:
    tooltip = 'Click to view station info'

    def add_marker(map_obj, station_code, marker_color):

        #Add popup text:
        popup=folium.Popup("""<meta content="text/html; charset=UTF-8"><style>td{padding: 3px;}</style><table>"""+
                           '<tr><td>Name: </td><td><b><a href="'+str(stations_df.station.loc[stations_df.stationId==station_code].values[0])+'"target="_blank">'+str(stations_df.stationName.loc[stations_df.stationId==station_code].values[0])+'</a></b></td></tr>'+
                           '<tr><td>Code:</td><td><b>'+station_code+'</b></td></tr>'+
                           '<tr><td>Sampling height(s):</td><td><b>'+", ". join(str(e) for e in sorted(stations_df.samplingheight.loc[stations_df.stationId==station_code].unique().astype(np.float32)))+'</b></td></tr>'+
                           '<tr><td>Country:</td><td><b>'+get_country_fullname_from_iso3166_2char(stations_df.country.loc[stations_df.stationId==station_code].values[0])+'</b></td></tr>'+
                           '<tr><td>Latitude:</td><td><b>'+str(stations_df.lat.loc[stations_df.stationId==station_code].values[0])+'</b></td></tr>'+
                           '<tr><td>Longitude:</td><td><b>'+str(stations_df.lon.loc[stations_df.stationId==station_code].values[0])+'</b></td></tr>'+
                           '<tr><td>PI:</td><td><b>'+str(stations_df.firstName.loc[stations_df.stationId==station_code].values[0])+' '+str(stations_df.lastName.loc[stations_df.stationId==station_code].values[0])+'</b></td></tr>'+
                           '<tr><td>Contact:</td><td><b>'+str(stations_df.email.loc[stations_df.stationId==station_code].values[0])+'</b></td></tr>'+
                           '</td></tr></table>',
                           max_width=450)

        #Create marker and add it to the map:
        folium.Marker(location=[float(stations_df.lat.loc[stations_df.stationId==station_code].values[0]),
                                float(stations_df.lon.loc[stations_df.stationId==station_code].values[0])],
                      popup=popup,
                      icon=folium.Icon(color=marker_color, icon=d_icon),
                      tooltip=tooltip).add_to(map_obj)


    #Get list of stations (not incl. selected station):
    station_ls = [i for i in stations_df.stationId.values if i!=selected_station]  

    #Create markers for all stations except selected station:
    for st in station_ls:
        add_marker(m, st, icon_col) 

    #Add marker for selected station:
    add_marker(m, selected_station, 'darkred')

    #Show map:
    display(m)


<a id='update_plot_funcs'></a>
<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>
<br>
<br>

## 6. Update plot functions
This part includes functions that update the data displayed in interactive plots.

In [ ]:
def update_icos_single_station_plot_binary(data_obj_id_ls, station, tracer, color, st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri Apr 07 10:05:00 2019
    Last Changed:     Tue Nov 17 10:30:00 2020
    Version:          0.1.3
    Author(s):        Karolina Pantazatou
    
    Description:      Function that gets the user's selection of station, tracer and color as input parameters, 
                      accesses and reads in the corresponding datafiles and outputs a Bokeh plot with observations
                      for the selected tracer.
                      
    Input parameters: 1. list of data object IDs
                         (var_name: 'data_obj_id_ls', var_type: List of Strings)
                      2. list of station info, containing 3-character station code and sampling height,
                         e.g. ['GAT', '30.0'] 
                         (var_name: 'station', var_type: List of Strings)
                      3. gas/traser, e.g. 'co', 'co2' or 'ch4'
                         (var_name: 'tracer', var_type: String)
                      4. color in hexadecimal code (var_name: 'color', var_type: String)
                      5. list of station objects (var_name: 'st_obj_ls', var_type: List)

    Output:           Bokeh Plot
    
    """
    
    #import modules:
    from bokeh.layouts import column
    from icoscp.cpb.dobj import Dobj
    
    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create dict to store the station info:
    station_info_dict = {}
    
    #Create a file object from the 1st object in the data object id list:  
    file = Dobj(data_obj_id_ls[0])
    var_row = file.variables.loc[file.variables.name == tracer]
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = var_row.type.values[0]
    
    #Get tracer unit:
    tracer_info_dict['tracer_unit'] = var_row.unit.values[0]
    
    #Loop through station obj list:
    for st in st_obj_ls:
        
        #Check if the station ID of the current station obj
        #is the same as the selected station ID
        if st.stationId==station[0]:
            
            #Get station info:
            station_info_dict['station_name'] = st.name
            station_info_dict['station_code'] = station[0]
            station_info_dict['station_sampling_height'] = station[1]
            station_info_dict['station_country_code'] = st.country
            station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(st.country)
            station_info_dict['station_lat'] = str(st.lat)
            station_info_dict['station_lon'] = str(st.lon)

    #Create list to store the data dataframes of all data object IDs:
    data_df_ls = []
    
    #Loop through every data object ID in the list:
    for dobjid in data_obj_id_ls: 
        
        #Get a pandas dataframe with all the columns for the selected data-object id:
        obs_data_df = Dobj(dobjid).data 
        
        #Add data dataframe of the current data object ID to the list:
        data_df_ls.append(obs_data_df)
        
    #Concatenate data dataframes to one dataframe:
    data_df = pd.concat(data_df_ls)
    
    #Sort the dataframe index in ascending order:
    data_df.sort_index(inplace=True)
    
    
    ### Plot ###
    #Plot station:
    p = plot_icos_single_station_binary(data_df,
                                        station_info_dict,
                                        tracer_info_dict,
                                        color=color)

    #Return plot:
    return p

In [ ]:
def update_exploring_multiple_tracers_binary(selection_list, st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Wed Nov 18 16:30:00 2020
    Version:          0.1.3
    Author(s):        Karolina Pantazatou
    
    Description:      Function that takes as input a list of sublists, where every sublist contains
                      a data object ID, a tracer string and a station ID string.
                      The function reads the corresponding ICOS Level-2 Atmospheric Tracer data files
                      for every sublist into sepparate pandas dataframes. Every data file produces two 
                      sepparate pandas dataframes; metadata dataframe and data dataframe.
                      These dataframes are then set as input parameters to a plot function, 
                      that returns a Bokeh Figure (plot).
                      The Bokeh Figure is then returned as output.
                      
                      
    Input parameters: 1. List with sublists of data object IDs & tracers
                         (var_name: 'selection_list', var_type: List)
                      2. List of ICOS atmospheric station objects
                         (var_name: 'st_obj_ls', var_type: List)

    Output:           Bokeh Plot
    
    """
    
    #import modules:
    from bokeh.layouts import column
    from icoscp.cpb.dobj import Dobj
    
    #Create lists to store lists of metadata & data dataframes for every station-tracer combination:
    co2_df_list = []
    co_df_list = []
    ch4_df_list = []
    
    #Create dict to store the station info:
    station_info_dict = {}
    
    
    #Loop through station obj list:
    for st in st_obj_ls:
        
        #Check if the station ID of the current station obj
        #is the same as the selected station ID
        if(st.stationId==selection_list[0][2]):
            
            #Get station info:
            station_info_dict['station_name'] = st.name
            station_info_dict['station_code'] = selection_list[0][2]
            station_info_dict['station_sampling_height'] = selection_list[0][3]
            station_info_dict['station_country_code'] = st.country
            station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(st.country)
            station_info_dict['station_lat'] = str(st.lat)
            station_info_dict['station_lon'] = str(st.lon)
            
            
    #Loop throught list of data-object-IDs and tracers and download and read file/s with ICOS data:
    for selection in selection_list:
        
        #Check tracer type:
        if(selection[1]=='co2'):
            
            #Create dictionary to store tracer info:
            tracer_info_dict_co2 = {}
            
            #Create a file object from the current data object id:
            file_co2 = Dobj(selection[0])  
            var_row_co2 = file_co2.variables.loc[file_co2.variables.name == selection[1]]
            
            
            #Get the tracer description:
            tracer_info_dict_co2['tracer_info'] = var_row_co2.type.values[0]

            #Get tracer unit:
            tracer_info_dict_co2['tracer_unit'] = var_row_co2.unit.values[0]
            
            
            #Add data dataframe for the current data object ID to the list:
            co2_df_list.append(file_co2.data)
            
            
        elif(selection[1]=='co'):
            
            #Create dictionary to store tracer info:
            tracer_info_dict_co = {}
            
            #Create a file object from the current data object id:
            file_co = Dobj(selection[0])  
            var_row_co = file_co.variables.loc[file_co.variables.name == selection[1]]
    
            #Get the tracer description:
            tracer_info_dict_co['tracer_info'] = var_row_co.type.values[0]
    
            #Get tracer unit:
            tracer_info_dict_co['tracer_unit'] = var_row_co.unit.values[0]
            
            #Add data dataframe for the current data object ID to the list:
            co_df_list.append(file_co.data)
            
            
        elif(selection[1]=='ch4'):
            
            #Create dict to store the tracer info:
            tracer_info_dict_ch4 = {}
            
            #Create a file object from the current data object id:
            file_ch4 = Dobj(selection[0])  
            var_row_ch4 = file_ch4.variables.loc[file_ch4.variables.name == selection[1]]
            
            #Get the tracer description:
            tracer_info_dict_ch4['tracer_info'] = var_row_ch4.type.values[0]

            #Get tracer unit:
            tracer_info_dict_ch4['tracer_unit'] = var_row_ch4.unit.values[0]
            
            #Add data dataframe for the current data object ID to the list:
            ch4_df_list.append(file_ch4.data)
        
        
        #If tracer is not one of the following: "CO2", "CO" or "CH4":
        else:
            print("\033[0;31;1m "+'Error! No support for this tracer!'+"\033[0;31;0m\n\n")
        
        
    #Create list to store lists of metadata & data dataframes for every station-tracer combination:
    info_list = []
        
    #Add all non-empty co2 lists to df_list:
    if(len(co2_df_list)>0):
        
        #Concatenate data dataframes to one dataframe:
        co2_data_df = pd.concat(co2_df_list)

        #Sort the dataframe index in ascending order:
        co2_data_df.sort_index(inplace=True)
            
        #Add the station_info dict, tracer_info dict and data dataframe to a list:
        info_list.append([station_info_dict, tracer_info_dict_co2, co2_data_df])
         
    #Add all non-empty tracer lists to df_list:
    if(len(co_df_list)>0):
        
        #Concatenate data dataframes to one dataframe:
        co_data_df = pd.concat(co_df_list)

        #Sort the dataframe index in ascending order:
        co_data_df.sort_index(inplace=True)
            
        #Add the pair of dataframes as a list to the list of dataframes:
        info_list.append([station_info_dict, tracer_info_dict_co, co_data_df])

    #Add all non-empty tracer lists to df_list:
    if(len(ch4_df_list)>0):
        
        #Concatenate data dataframes to one dataframe:
        ch4_data_df = pd.concat(ch4_df_list)

        #Sort the dataframe index in ascending order:
        ch4_data_df.sort_index(inplace=True)
            
        #Add the pair of dataframes as a list to the list of dataframes:
        info_list.append([station_info_dict, tracer_info_dict_ch4, ch4_data_df])

        
    #Call a function to plot data from all dataframes in the list of dataframes:
    p = plot_icos_single_station_multiple_tracers_binary(info_list)

    
    #Set output channel:
    output_notebook()
    
    #Show plot:
    show(p)

In [ ]:
def update_exploring_multiple_stations_binary(station_dobj_ls, st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Wed Nov 18 17:30:00 2020
    Version:          0.1.3
    Author(s):        Karolina Pantazatou
    
    Description:      Function that takes as input a list of sublists, where every sublist contains
                      one or more lists with the ICOS Atmospheric Level-2 data-object-ID, the station
                      code, the sampling height and the selected tracer for a selected station and
                      sampling height.
                      More than one lists per selected station, sampling height and tracer can occur
                      for ICOS Level-2 data in cases where the measuring equipment has changed (i.e.
                      installation of new instrument with new instrument ID). When a new instrument
                      is used, the measurments from this instrument will be put in a different file,
                      which in turns corresponds to a new data object ID.
                      The function reads in the data from the data file corresponding to every data
                      object ID and stores it in metadata- and data-dataframes. One set of metadata-
                      and data-dataframes are produced for every file. The function checks if there
                      are more than one files related to the same station with the same sampling
                      height and tracer, and if this is the case, it concatenates the data-dataframes.
                      All sets of metadata- and data-dataframes are then stored in sublists of a list
                      and sent as input parameters to a plot function, that returns a Bokeh Figure (plot).
                      The Bokeh Figure is then returned as output.
                      
                      
    Input parameters: 1. List of sublists with list(s) including info about an ICOS Atmospheric Level-2
                         data-object-ID, the station code, the sampling height and the selected tracer
                         (var_name: 'station_dobj_ls', var_type: List)
                      2. List of ICOS atmospheric station objects
                         (var_name: 'st_obj_ls', var_type: List)

    Output:           Bokeh Plot
    
    """
    
    #import modules:
    from bokeh.layouts import column
    from icoscp.cpb.dobj import Dobj
    
    #Create list to store the metadata & data dataframes for all combinations of tracers and stations:
    station_df_ls = []
    
    
    #Loop through every data object ID in the list:
    for station_ls in station_dobj_ls:
        
        #Create dictionary to store tracer info:
        tracer_info_dict = {}
        
        #Create dict to store the station info:
        station_info_dict = {}
        
        #Create a file object from the current data object id:
        file = Dobj(station_ls[0][0]) 
        var_row = file.variables.loc[file.variables.name == station_ls[0][1]]
    
        #Get the tracer description:
        tracer_info_dict['tracer_info'] = var_row.type.values[0]
    
        #Get tracer unit:
        tracer_info_dict['tracer_unit'] = var_row.unit.values[0]

        #Loop through station obj list:
        for st in st_obj_ls:

            #Check if the station ID of the current station obj
            #is the same as the selected station ID
            if st.stationId==station_ls[0][2]:

                #Get station info:
                station_info_dict['station_name'] = st.name
                station_info_dict['station_code'] = station_ls[0][2]
                station_info_dict['station_sampling_height'] = station_ls[0][3] 
                station_info_dict['station_country_code'] = st.country
                station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(st.country)
                station_info_dict['station_lat'] = str(st.lat)
                station_info_dict['station_lon'] = str(st.lon)

        #Create list to store the data dataframes of all data object IDs for the same station,
        #tracer and sampling height:
        df_ls = []

        #station_info[0] --- > dataobjid (e.g. 'MdYIndlCMyEp2BoGwUL_0Jqq')
        #station_info[1] --- > Tracer (e.g. 'co2')
        #station_info[2] --- > Station Code (e.g. 'HPB') 
        #station_info[3] --- > Sampling Height (e.g. '50.0')
        
        #Download data from all files that correspond to the same station,
        #at the same sampling height and include data for the same tracer
        #in metadata- and data- dataframes that are stored as sublists in a list:
        df_ls = [Dobj(station_info[0]).data for station_info in station_ls]

        #Concatenate the data-dataframes that include tracer data for
        #the same station at the same sampling height, to one data-dataframe:
        data_df = pd.concat([df for df in df_ls])
        data_df.sort_index(inplace=True)
       

        #Append metadata-dataframe and concatenated data-dataframe to list:
        station_df_ls.append([station_info_dict, tracer_info_dict, data_df])

        
    #Get plot:
    p = plot_icos_single_tracer_multiple_stations_binary(station_df_ls) 
    
    #Output should be in the notebook
    output_notebook()
    
    #Show plot
    show(p, notebook_handle=True)
   

In [ ]:
def update_focus_plot_binary(data_obj_id_ls, station, tracer, color, st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Wed Nov 18 18:15:00 2020
    Version:          0.1.3
    Author(s):        Karolina Pantazatou
    
    Description:      Function that takes as input a list of ICOS data object IDs, the station name,
                      the tracer type and the plot color. The function reads the corresponding
                      ICOS Level-2 Atmospheric Tracer data files for every data object ID into sepparate
                      pandas dataframes. Every data file produces two sepparate pandas dataframes;
                      metadata dataframe and data dataframe. These dataframes are then set as input
                      parameters to a plot function, that returns a Bokeh Figure (plot).
                      The Bokeh Figure is then returned as output.
                      
                      
    Input parameters: 1. List with sublists of data object IDs
                         (var_name: 'data_obj_id_ls', var_type: List)
                      2. ICOS Station 3-charcter Code
                         (var_name: 'station', var_type: String)
                      3. Tracer/gas, e.g. 'co2'
                         (var_name: 'tracer', var_type: String)
                      4. Plot Color
                         (var_name: 'color', var_type: String)
                      5. List with ICOS atmospheric station objects
                         (var_name: 'st_obj_ls', var_type: List)
                      

    Output:           Bokeh Plot
    
    """
      
    #import modules:
    from icoscp.cpb.dobj import Dobj
    from bokeh.layouts import column
    from bokeh.io import push_notebook, output_notebook
    from bokeh.plotting import show
    
    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create dict to store the station info:
    station_info_dict = {}
    
    #Create a file object from the 1st object in the data object id list:
    file = Dobj(data_obj_id_ls[0])  
    var_row = file.variables.loc[file.variables.name == tracer]
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = var_row.type.values[0]
    
    #Get tracer unit:
    tracer_info_dict['tracer_unit'] = var_row.unit.values[0]
    
    #Loop through station obj list:
    for st in st_obj_ls:
        
        #Check if the station ID of the current station obj
        #is the same as the selected station ID
        if st.stationId==station[0]:
            
            #Get station info:
            station_info_dict['station_name'] = st.name
            station_info_dict['station_code'] = station[0]
            station_info_dict['station_sampling_height'] = station[1]
            station_info_dict['station_country_code'] = st.country
            station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(st.country)
            station_info_dict['station_lat'] = str(st.lat)
            station_info_dict['station_lon'] = str(st.lon)

    
    #Create list to store the data dataframes of all data object IDs:
    data_df_ls = []
    
    #Loop through every data object ID in the list:
    for dobjid in data_obj_id_ls: 
        
        #Get data dataframe:
        obs_data_df = Dobj(dobjid).data
        
        #Add data dataframe corresponding to the current data object ID to the list:
        data_df_ls.append(obs_data_df)
        
    #Concatenate data dataframes to one dataframe:
    data_df = pd.concat(data_df_ls)
    
    #Sort the dataframe index in ascending order:
    data_df.sort_index(inplace=True)
    
    
    ### PLOT ###
    #Plot station:
    p1, p2 = plot_icos_focus_binary(data_df, station_info_dict, tracer_info_dict, tracer, color)
    
    #Set the output to be organized columnwise (i.e. output plots one under the other):
    layout = column(p1,p2)
    
    #Set the notebook as the prefered output channel:
    output_notebook()
    
    #Show plot:
    show(layout)

    #Update plot:
    #push_notebook()

In [ ]:
def update_basic_statistics_binary(station_dobj_ls, start_date, end_date, st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Wed May 15 10:30:00 2018
    Last Changed:     Wed Nov 18 18:30:00 2020
    Version:          0.1.3
    Author(s):        Karolina Pantazatou
    
    Description:      Function that takes as input a list of sublists, where every sublist contains
                      one or more lists with the ICOS Atmospheric Level-2 data-object-ID, the station
                      code, the sampling height and the selected tracer for a selected station and
                      sampling height. The Function also takes as input the selected start-date and
                      end-date.
                      More than one lists per selected station, sampling height and tracer can occur
                      for ICOS Level-2 data in cases where the measuring equipment has changed (i.e.
                      installation of new instrument with new instrument ID). When a new instrument
                      is used, the measurments from this instrument will be put in a different file,
                      which in turns corresponds to a new data object ID.
                      The function reads in the data from the data file corresponding to every data
                      object ID and stores it in metadata- and data-dataframes. One set of metadata-
                      and data-dataframes are produced for every file. The function checks if there
                      are more than one files related to the same station with the same sampling
                      height and tracer, and if this is the case, it concatenates the data-dataframes.
                      The data dataframe is filtered to only contain data between the selected start-date
                      och end-date.
                      All sets of metadata- and data-dataframes are then stored in sublists of a list
                      and sent as input parameters to a function that will compute the basic statistics
                      for the tracer-column of every data-dataframe. The dataframe including the 
                      basic statistics results for every selected station are the returned as output. 
                      
                      
    Input parameters: 1. List of sublists with list(s) including info about an ICOS Atmospheric Level-2
                         data-object-ID, the station code, the sampling height and the selected tracer
                         (var_name: 'station_dobj_ls', var_type: List)
                      2. Start date - user's input
                         (var_name: 'start_date', var_type: DateTime Object)
                      3. End date - user's input
                         (var_name: 'end_date', var_type: DateTime Object)
                      4. List of ICOS atmospheric station objects
                         (var_name: 'st_obj_ls', var_type: List)
                      

    Output:           Pandas DataFrame
    
    """
    
    #Import modules:
    import pandas as pd
    from icoscp.cpb.dobj import Dobj
    
 
    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create a file object from the 1st object in the data object id list:
    file = Dobj(station_dobj_ls[0][0][0])  
    var_row = file.variables.loc[file.variables.name == station_dobj_ls[0][0][1]]
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = var_row.type.values[0]
    
    #Get tracer unit:
    tracer_info_dict['tracer_unit'] = var_row.unit.values[0]
    
    #Get pandas dataframe with all ICOS stations:
    icos_stations_df = RunSparql(sparql_query=sparqls.get_coords_icos_stations_atc(), output_format='pandas').run()
    
    #Create list to store the metadata & data dataframes for all combinations of tracers and stations:
    station_df_ls = []

    
    
    #Loop through every data object ID in the list:
    for station_ls in station_dobj_ls:
        
        #Create a dictionary to store all station info:
        station_info_dict = {}
    
        #Loop through station obj list:
        for st in st_obj_ls:

            #Check if the station ID of the current station obj
            #is the same as the selected station ID
            if st.stationId==station_ls[0][2]:

                #Get station info:
                station_info_dict['station_name'] = st.name
                station_info_dict['station_code'] = station_ls[0][2]
                station_info_dict['station_sampling_height'] = station_ls[0][3]
                #station_info_dict['station_country_code'] = st.country
                #station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(st.country)
                #station_info_dict['station_lat'] = str(st.lat)
                #station_info_dict['station_lon'] = str(st.lon)

    
        #Create list to store the data dataframes of all data object IDs
        #for all stations:
        df_ls = []

        #station_info[0] --- > dataobjid (e.g. 'MdYIndlCMyEp2BoGwUL_0Jqq')
        #station_info[1] --- > Tracer (e.g. 'co2')
        #station_info[2] --- > Station Code (e.g. 'HPB')
        #station_info[3] --- > Sampling Height (e.g. '50.0')
        

        #Download data from all files that correspond to the same station,
        #at the same sampling height and include data for the same tracer
        #in a pandas dataframe that are then stored as sublists in a list:
        df_ls = [Dobj(station_info[0]).data
                 for station_info in station_ls]

        #Concatenate the data-dataframes that include tracer data for
        #the same station at the same sampling height, to one data-dataframe:
        data_df = pd.concat([df for df in df_ls])
        
        #Add column with datetime object:
        data_df['DateTime'] = pd.to_datetime(data_df['TIMESTAMP'], unit='ms') 

        #Create a copy of the dataframe and set "DateTime" as index:
        data_df_ind = data_df.copy().set_index('DateTime')

        #Sort the dataframe index in ascending order:
        data_df_ind.sort_index(inplace=True)
        
        
        #Filter dataframe to only contain data between the selected dates:
        data_df_filt = data_df_ind.loc[start_date:end_date]

        #Append metadata-dataframe and concatenated data-dataframe to list:
        station_df_ls.append([data_df_filt, station_info_dict, tracer_info_dict])

    #Call function to compute basic statistics for all selected stations
    #and return the result:
    return calculate_basic_statistics_binary(station_df_ls, station_dobj_ls[0][0][1])



In [ ]:
def update_corr_stat_multi_binary(station_dobj_ls, start_date, end_date, st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Wed Nov 18 19:00:00 2020
    Version:          0.1.3
    Author(s):        Karolina Pantazatou
    
    Description:      Function that takes as input a list of sublists, where every sublist contains an
                      ICOS Data Object ID and a tracer string. The function reads the corresponding
                      ICOS Level-2 Atmospheric Tracer data files for every data object ID into sepparate
                      pandas dataframes. Every data file produces two sepparate pandas dataframes;
                      metadata dataframe and data dataframe. The data dataframe is filtered to only include
                      values for the time period that was selected by the user.
                      Then the correlation is computed between every station and the results are returned
                      in a pandas dataframe.
                                        
    Input parameters: 1. List with sublists of data object IDs and tracers
                         (var_name: 'station_dobj_ls', var_type: List)
                      2. Start date - user's input
                         (var_name: 'start_date', var_type: DateTime Object)
                      3. End date - user's input
                         (var_name: 'end_date', var_type: DateTime Object)
                      4. List of ICOS atmospheric station objects
                         (var_name: 'st_obj_ls', var_type: List)
                      
    Output:           Pandas DataFrame
    
    """
    
    #Import modules:
    import pandas as pd
    from icoscp.cpb.dobj import Dobj
    

    #Add dictionary to transform digits to subscript:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    
    
    #Create list to store the metadata & data dataframes for all combinations of tracers and stations:
    station_df_ls = []

    
    #Loop through every data object ID in the list:
    for station_ls in station_dobj_ls:
        
        #Create dictionary to store tracer info:
        tracer_info_dict = {}
        
        #Create a dictionary to store all station info:
        station_info_dict = {}
        
        #Create list to store the data dataframes of all data object IDs for :
        df_ls = []
        
        #Create a file object from the 1st object in the data object id list:
        file = Dobj(station_ls[0][0])  
        var_row = file.variables.loc[file.variables.name == station_ls[0][1]]
    
        #Get the tracer description:
        tracer_info_dict['tracer_info'] = var_row.type.values[0]

        #Get tracer unit:
        tracer_info_dict['tracer_unit'] = var_row.unit.values[0]

        #Loop through station obj list:
        for st in st_obj_ls:

            #Check if the station ID of the current station obj
            #is the same as the selected station ID
            if st.stationId==station_ls[0][2]:

                #Get station info:
                station_info_dict['station_name'] = st.name
                station_info_dict['station_code'] = station_ls[0][2]
                station_info_dict['station_sampling_height'] = station_ls[0][3]

    

        #station_info[0] --- > dataobjid (e.g. 'MdYIndlCMyEp2BoGwUL_0Jqq')
        #station_info[1] --- > Station Code (e.g. 'HPB')
        #station_info[2] --- > Sampling Height (e.g. '50.0')
        #station_info[3] --- > Tracer (e.g. 'co2')

        #Download data from all files that correspond to the same station,
        #at the same sampling height and include data for the same tracer
        #in metadata- and data- dataframes that are stored as sublists in a list:
        df_ls = [Dobj(station_info[0]).data for station_info in station_ls]
        
        #Check size of df_ls:
        if(len(df_ls)>1):
            #Concatenate the data-dataframes that include tracer data for
            #the same station at the same sampling height, to one data-dataframe:
            data_df = pd.concat([df for df in df_ls])
            
        else:
            data_df = df_ls[0]
            
            
        #Add column with datetime object:
        data_df['DateTime'] = pd.to_datetime(data_df['TIMESTAMP'], unit='ms') 

        #Create a copy of the dataframe and set "DateTime" as index:
        data_df_ind = data_df.copy().set_index('DateTime')

        #Sort the dataframe index in ascending order:
        data_df_ind.sort_index(inplace=True)
            
        #Filter dataframe to only contain data between the selected dates:
        data_df_filt = data_df_ind.loc[start_date:end_date]
        
        #Append metadata-dataframe and concatenated data-dataframe to list:
        station_df_ls.append([data_df_filt, station_info_dict, tracer_info_dict])
    
    
    #Extract the tracer-column from every station's data dataframe to a new pandas dataframe:
    tracer_df_ls =[pd.DataFrame({station_df_ls[i][1]['station_code']+'_'+
                                 station_df_ls[i][1]['station_sampling_height']+' ('+
                                 station_df_ls[i][2]['tracer_info'].replace(' mixing ratio (dry mole fraction)', '').translate(SUB)+
                                 ')': station_df_ls[i][0][station_df_ls[i][2]['tracer_info'].replace(' mixing ratio (dry mole fraction)', '').lower()]})
                   for i in range(len(station_df_ls))]
    
    #Concatenate dataframes to one final dataframe:
    tracers_df = pd.concat(tracer_df_ls, axis=1)
    
    #Get correlation between data:
    corr_df = tracers_df.corr(method='pearson')
    
    #Return dataframe:
    return corr_df
    


In [ ]:
#Function that updates the plot every time the user interacts with a widget:
def update_smoothing_plot_binary(data_obj_id_ls, station, tracer, days, color, st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Wed Nov 18 19:15:00 2020
    Version:          0.1.3
    Author(s):        Karolina Pantazatou
    
    
    Description:      Function that takes as input an ICOS data object ID, the station name,
                      the tracer type, the total number of days over which to average the
                      ICOS tracer data and the plot color. The function reads the corresponding
                      ICOS Level-2 Atmospheric Tracer data files for every data object ID into sepparate
                      pandas dataframes. Every data file produces two sepparate pandas dataframes;
                      metadata dataframe and data dataframe. The values of the tracer in the data
                      dataframe are then averaged by a window that corresponds to the related input
                      parameter. The updated data dataframe and the metadata dataframe are then set as input
                      parameters to a plot function, that returns a Bokeh Figure (plot).
                      The Bokeh Figure is then returned as output.
                      
                      
    Input parameters: 1. List with sublists of data object IDs
                         (var_name: 'data_obj_id_ls', var_type: List)
                      2. ICOS Station 3-charcter Code
                         (var_name: 'station', var_type: String)
                      3. Tracer/gas, e.g. 'co2'
                         (var_name: 'tracer', var_type: String)
                      4. Number of days to average by
                         (var_name: 'days', var_type: Integer)
                      5. Plot Color
                         (var_name: 'color', var_type: String)
                      6. List of ICOS atmospheric station objects
                         (var_name: 'st_obj_ls', var_type: List)
       
       
    Output:           Bokeh Plot
    
    """
    
    #import modules:
    import pandas as pd
    from bokeh.layouts import column
    from icoscp.cpb.dobj import Dobj
    
    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create a dictionary to store the station info:
    station_info_dict = {}
    
    #Create a file object from the 1st object in the data object id list:
    file = Dobj(data_obj_id_ls[0])  
    var_row = file.variables.loc[file.variables.name == tracer]
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = var_row.type.values[0]
    
    #Get tracer unit:
    tracer_info_dict['tracer_unit'] = var_row.unit.values[0]

    #Loop through station obj list:
    for st in st_obj_ls:
        
        #Check if the station ID of the current station obj
        #is the same as the selected station ID
        if st.stationId==station[0]:
            
            #Get station info:
            station_info_dict['station_name'] = st.name
            station_info_dict['station_code'] = station[0]
            station_info_dict['station_sampling_height'] = station[1]
            station_info_dict['station_country_code'] = st.country
            station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(st.country)
            station_info_dict['station_lat'] = str(st.lat)
            station_info_dict['station_lon'] = str(st.lon)

    #Create list to store the data dataframes of all data object IDs:
    data_df_ls = []
    
    #Loop through every data object ID in the list:
    for dobjid in data_obj_id_ls: 
        
        #Get all the columns for the selected dataobject id:
        obs_data_df = Dobj(dobjid).data
        
        #Add data dataframe of the current data object ID to the list:
        data_df_ls.append(obs_data_df)
        
    #Concatenate data dataframes to one dataframe:
    data_df = pd.concat(data_df_ls)
    
    #Add column with datetime object:
    data_df['DateTime'] = pd.to_datetime(data_df['TIMESTAMP'], unit='ms') 
    
    #Create a copy of the dataframe and set "DateTime" as index:
    data_df_ind = data_df.copy().set_index('DateTime')
    
    #Sort the dataframe index in ascending order:
    data_df_ind.sort_index(inplace=True)
    
    

    #Smoothing the glyph-line, by using a window of a selected number of days that averages the values:
    if days == 0:
        data_df_ind['rolling_mean'] = data_df_ind[tracer]
    
    #If the number of days is higher than zero:
    else:
        
        #If number of days is an even number
        if(days%2==0):
            data_df_ind['rolling_mean'] = data_df_ind[tracer].rolling('{0}D'.format(days), closed='left').mean().shift(int(-days/2)*24, freq='h')
        
        #If number of days is an odd number:
        else:
            data_df_ind['rolling_mean'] = data_df_ind[tracer].rolling('{0}D'.format(days), closed='left').mean().shift(round(-days/2)*24, freq='h')
            
    
    #Plot station:
    p = plot_icos_single_station_smoothing_binary(data_df_ind, station_info_dict, tracer_info_dict, tracer, color)
    
    #Output should be in the notebook
    output_notebook()
    
    #Show plot
    show(p, notebook_handle=True)  


In [ ]:
def update_comparing_binary(station_dobj_ls, num_of_stations, st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Wed Nov 18 19:40:00 2020
    Version:          0.1.3
    Author(s):        Karolina Pantazatou
    
    Description:      Function that takes as input a list of sublists, where every sublist contains an
                      ICOS Data Object ID and a tracer string. The function aslo takes as input an integer
                      representing the total number of stations selected. This parameter is then used to get
                      a sublist of the same number of different colors from a colormap. The function reads
                      the corresponding ICOS Level-2 Atmospheric Tracer data files for every data object ID
                      into sepparate pandas dataframes. Every data file produces two sepparate pandas dataframes;
                      metadata dataframe and data dataframe. Then, every set of metadata- and data-dataframes
                      are sent as input parameters to a plot function, that will produce a sepparate plot
                      for every station.
                      
                      
    Input parameters: 1. List with sublists of data object IDs and tracers
                         (var_name: 'station_dobj_ls', var_type: List)
                         
                      2. Total number of stations selected
                         (var_name: 'num_of_stations', var_type: Integer)
                         
                      3. List of ICOS atmospheric station objects
                         (var_name: 'st_obj_ls', var_type: List)
                      

    Output:           Interactive Bokeh Plot(s)
    
    """
    
    #import modules:
    from bokeh.layouts import column
    from bokeh.io import push_notebook, output_notebook
    from bokeh.plotting import show
    from icoscp.cpb.dobj import Dobj
    
    #Get colormap:
    colormap = get_colormap(num_of_stations)
    
    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create dict to store the station info:
    station_info_dict = {}
    
    #Create a file object from the 1st object in the data object id list:
    file = Dobj(station_dobj_ls[0][0][0])  
    var_row = file.variables.loc[file.variables.name == station_dobj_ls[0][0][1]]
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = var_row.type.values[0]
    
    #Get tracer unit:
    tracer_info_dict['tracer_unit'] = var_row.unit.values[0]

    #Define and initialize list to store station plots:
    plot_list = []
    
    #Add counter:
    counter = 0

    #Loop through every data object ID in the list:
    for station_ls in station_dobj_ls:

        #Loop through station obj list:
        for st in st_obj_ls:

            #Check if the station ID of the current station obj
            #is the same as the selected station ID
            if st.stationId==station_ls[0][2]:

                #Get station info:
                station_info_dict['station_name'] = st.name
                station_info_dict['station_code'] = station_ls[0][2]
                station_info_dict['station_sampling_height'] = station_ls[0][3]
                station_info_dict['station_country_code'] = st.country
                station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(st.country)
                station_info_dict['station_lat'] = str(st.lat)
                station_info_dict['station_lon'] = str(st.lon)


        #Create list to store the data dataframes of all data object IDs for :
        df_ls = []

        #station_info[0] --- > dataobjid (e.g. 'MdYIndlCMyEp2BoGwUL_0Jqq')
        #station_info[1] --- > Tracer (e.g. 'co2')
        #station_info[2] --- > Station Code (e.g. 'HPB')
        #station_info[3] --- > Sampling Height (e.g. '50.0')
        
        
        #Download data from all files that correspond to the same station,
        #at the same sampling height and include data for the same tracer
        #in metadata- and data- dataframes that are stored as sublists in a list:
        df_ls = [Dobj(station_info[0]).data
                 for station_info in station_ls]

        #Concatenate the data-dataframes that include tracer data for
        #the same station at the same sampling height, to one data-dataframe:
        data_df = pd.concat([df for df in df_ls])
        #data_df.sort_index(inplace=True)
        
        #Add plot-object to list:
        plot_list.append(plot_icos_single_station_binary(data_df,
                                                         station_info_dict,
                                                         tracer_info_dict,
                                                         color=colormap[counter]))
        
        #Increase counter:
        counter = counter + 1
    
    #Organize the plots in a "column" layout:
    layout = column(plot_list)
    
    #Output will be displayed in notebook:
    output_notebook()
    
    #Show plot/s:
    show(layout, notebook_handle=True)


In [ ]:
def update_icos_single_station_plot_L1_L2_binary(data_obj_id_L1_ls, data_obj_id_L2_ls, station_code, station_sampl_height, tracer, color, st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Wed Nov 18 19:55:00 2020
    Version:          0.1.3
    Author(s):        Karolina Pantazatou
    
    Description:      Function that takes as input a list of ICOS Level-1 data object IDs, a list
                      of ICOS Level-2 data object IDs, the tracer type and the plot color. The
                      function reads the corresponding ICOS Level-1 and Level-2 Atmospheric Tracer
                      data files for every data object ID into sepparate pandas dataframes. Every
                      data file produces two sepparate pandas dataframes; metadata dataframe and
                      data dataframe. These dataframes are then set as input parameters to a
                      plot function, that returns a Bokeh Figure (plot).
                      The Bokeh Figure is then returned as output.
                      
                      
    Input parameters: 1. List with sublists of ICOS Level-1 data object IDs
                         (var_name: 'data_obj_id_L1_ls', var_type: List)
                      2. List with sublists of ICOS Level-2 data object IDs
                         (var_name: 'data_obj_id_L2_ls', var_type: List)
                      3. Station 3-character code
                         (var_name: 'station_code', var_type: String)
                      4. Station sampling height, expressed as meters a.g.l.
                         (var_name: 'station_sampl_height', var_type: String)
                      5. Tracer/gas, e.g. 'co2'
                         (var_name: 'tracer', var_type: String)
                      6. Plot Color
                         (var_name: 'color', var_type: String)
                      7. List of ICOS atmospheric station objects
                         (var_name: 'st_obj_ls', var_type: List)

    Output:           Bokeh Plot
    
    """
    
    #Import modules:
    from icoscp.cpb.dobj import Dobj
    
    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create dict to store the station info:
    station_info_dict = {}
    
    #Create a file object from the 1st object in the data object id list:
    file = Dobj(data_obj_id_L1_ls[0])  
    var_row = file.variables.loc[file.variables.name == tracer]
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = var_row.type.values[0]
    
    #Get tracer unit:
    tracer_info_dict['tracer_unit'] = var_row.unit.values[0]

    #Loop through station obj list:
    for st in st_obj_ls:
        
        #Check if the station ID of the current station obj
        #is the same as the selected station ID
        if st.stationId==station_code:
            
            #Get station info:
            station_info_dict['station_name'] = st.name
            station_info_dict['station_code'] = station_code
            station_info_dict['station_sampling_height'] = station_sampl_height
            station_info_dict['station_country_code'] = st.country
            station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(st.country)
            station_info_dict['station_lat'] = str(st.lat)
            station_info_dict['station_lon'] = str(st.lon)

    
    
    
    #Create list to store the data dataframes of all data object IDs:
    L1_data_df_ls = []
    
    #Loop through every data object ID in the list:
    for dobjid_L1 in data_obj_id_L1_ls:
        
        #Get a pandas dataframe with all the columns for the selected data-object id:
        obs_data_df_L1 = Dobj(dobjid_L1).data
        
        #Add data dataframe of the current data object ID to the list:
        L1_data_df_ls.append(obs_data_df_L1)
        
    #Concatenate data dataframes to one dataframe:
    data_L1_df = pd.concat(L1_data_df_ls)
    
    #Add column with datetime object:
    data_L1_df['DateTime'] = pd.to_datetime(data_L1_df['TIMESTAMP'], unit='ms') 

    #Create a copy of the dataframe and set "DateTime" as index:
    data_df_ind_L1 = data_L1_df.copy().set_index('DateTime')

    #Sort the dataframe index in ascending order:
    data_df_ind_L1.sort_index(inplace=True)
        
    
    #Create list to store the data dataframes of all data object IDs:
    L2_data_df_ls = []
    
    #Loop through every data object ID in the list:
    for dobjid_L2 in data_obj_id_L2_ls:
        
        #Get a pandas dataframe with all the columns for the selected data-object id:
        obs_data_df_L2 = Dobj(dobjid_L2).data
        
        #Add data dataframe of the current data object ID to the list:
        L2_data_df_ls.append(obs_data_df_L2)
        
    #Concatenate data dataframes to one dataframe:
    data_L2_df = pd.concat(L2_data_df_ls)

   #Add column with datetime object:
    data_L2_df['DateTime'] = pd.to_datetime(data_L2_df['TIMESTAMP'], unit='ms') 

    #Create a copy of the dataframe and set "DateTime" as index:
    data_df_ind_L2 = data_L2_df.copy().set_index('DateTime')

    #Sort the dataframe index in ascending order:
    data_df_ind_L2.sort_index(inplace=True)
       
    #Plot station:
    p = plot_icos_single_station_L1_L2_binary([data_df_ind_L1, data_df_ind_L2],
                                              station_info_dict,
                                              tracer_info_dict,
                                              color)
    
    #Show plot
    show(p)
   

In [ ]:
def update_icos_single_station_plot_LX_binary(data_obj_id_LX_ls, station_code, station_sampl_height, tracer, color, level, st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Wed Nov 18 19:45:00 2020
    Version:          0.1.3
    Author(s):        Karolina Pantazatou
    
    Description:      Function that takes as input a list of ICOS Level-X data object IDs, the
                      tracer type, the plot color, the ICOS data level and a list of station objects.
                      The function reads the corresponding ICOS Level-1 Tracer data files for
                      every data object ID into sepparate pandas dataframes. Every data file
                      produces two sepparate pandas dataframes; metadata dataframe and data dataframe.
                      These dataframes are then set as input parameters to a plot function,
                      that returns a Bokeh Figure (plot).
                      The Bokeh Figure is then returned as output.
                      
                      
    Input parameters: 1. List with sublists of ICOS Level-X data object IDs
                         (var_name: 'data_obj_id_LX_ls', var_type: List)
                      2. Station Code
                         (var_name: 'station_code', var_type: String)
                      3. Station sampling height
                         (var_name: 'station_sampl_height', var_type: String)
                      4. Tracer/gas, e.g. 'co2'
                         (var_name: 'tracer', var_type: String)
                      5. Plot Color
                         (var_name: 'color', var_type: String)
                      6. ICOS Atmospheric Data Level
                         (var_name: 'level', var_type: Integer)
                      7. List with ICOS atmospheric station objects
                         (var_name: 'st_obj_ls', var_type: List)
                      

    Output:           Bokeh Plot
    
    """
    
    #Import modules:
    from icoscp.cpb.dobj import Dobj
    
    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create dict to store the station info:
    station_info_dict = {}
    
    #Create list to store the data dataframes of all data object IDs:
    data_df_ls = []
    
    #Create a file object from the 1st object in the data object id list:
    file = Dobj(data_obj_id_LX_ls[0])  
    var_row = file.variables.loc[file.variables.name == tracer]
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = var_row.type.values[0]
    
    #Get tracer unit:
    tracer_info_dict['tracer_unit'] = var_row.unit.values[0]
    
    #Loop through station obj list:
    for st in st_obj_ls:
        
        #Check if the station ID of the current station obj
        #is the same as the selected station ID
        if st.stationId==station_code:
            
            #Get station info:
            station_info_dict['station_name'] = st.name
            station_info_dict['station_code'] = station_code
            station_info_dict['station_sampling_height'] = station_sampl_height
            station_info_dict['station_country_code'] = st.country
            station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(st.country)
            station_info_dict['station_lat'] = str(st.lat)
            station_info_dict['station_lon'] = str(st.lon)

    
    #Loop through every data object ID in the list:
    for dobjid in data_obj_id_LX_ls: 
        
        #Get a pandas dataframe with all the columns for the selected dataobject id:
        obs_data_df = Dobj(dobjid).data
        
        #Add data dataframe of the current data object ID to the list:
        data_df_ls.append(obs_data_df)
        
    #Concatenate data dataframes to one dataframe:
    data_LX_df = pd.concat(data_df_ls)
    
        
    #Call plotting function:
    p = plot_icos_single_station_binary(data_LX_df, station_info_dict, tracer_info_dict, level, color)
    
    #Show plot
    show(p)


<a id='plotting_funcs'></a>
<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>
<br>
<br>

## 7. Plotting functions
This part includes functions that create interactive plots with Bokeh visualization library.

In [ ]:
def plot_icos_single_station_binary(df_data, station_info_dict, tracer_info_dict, level=2, color='#0F0C08'):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Mon Apr 07 09:30:00 2019
    Last Changed:     Mon Apr 07 09:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that plots the content of an ICOS Level-1 or Level-2 Atmospheric Data file to an 
                      interactive plot using the Bokeh interactive visualization library.
                      Bokeh (URL): https://bokeh.pydata.org/en/latest/
                      
    Input parameters: 1. ICOS Level-1 or Level-2 Atmospheric Observation Data
                         (var_name: 'df_data', var_type: pandas dataframe) 
                      2. Dictionary with station info
                         (var_name: 'station_info_dict', var_type: dictionary) 
                      3. Dictionary with tracer info
                         (var_name: 'tracer_info_dict', var_type: dictionary) 
                      4. Data level [optional]
                         (var_name: 'level', var_type: Integer)
                      5. Color for Line- or Circle Glyph [optional]
                         (var_name: 'color', var_type: String)
                     
    
    Default value for color: The default value for line-glyph or circle-glyph color is "lightblue".  
    
    Default value for level: The default value for data level is "2". Function calls for Level-2 data do not have
                             to include a value for the level input parameter. 
    
    Output:           Bokeh Figure Object (plot) 
    
    """
    
    #Import modules to create figure:
    import pandas as pd
    from bokeh.plotting import figure
    from bokeh.models import ColumnDataSource, HoverTool, Label, Legend
    from datetime import datetime

    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

    #Define Datasets:
    x = pd.to_datetime(df_data['TIMESTAMP'], unit='ms')
    y = df_data[tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].values #give tracer as parameter, where tracer can be ['co', 'co2']
    z = df_data["NbPoints"].values
    w = df_data["Stdev"].values
    o = df_data["Flag"].values
    #u = df_data["InstrumentId"].values

    #Create a ColumnDataSource object:
    source = ColumnDataSource( data = {'x':x, 'y':y, 'z':z, 'w':w, 'o':o,} )


    #Create a figure object:
    p = figure(plot_width=900,
               plot_height=400,
               x_axis_label='Time (UTC)', 
               y_axis_label=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB)+ ' (' +
               tracer_info_dict['tracer_unit'].translate(SUP) + ')',
               x_axis_type='datetime',
               title = tracer_info_dict['tracer_info'].translate(SUB)+'    '+
               station_info_dict['station_name'] +', '+
               station_info_dict['station_country'] +', '+
               station_info_dict['station_sampling_height'] +' m.a.g.l.',
               tools='pan,box_zoom,wheel_zoom,undo,redo,reset,save')

    #Create glyphs:
    g0 = p.circle('x','y', source=source, radius=.02, color=color)
    
    #If data is level-2 data:
    if(level==2):
        g1 = p.line('x','y', source=source, line_width=1, color=color, name=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB))
    
    #If data is level-1 data:
    else:
        g1 = p.line('x','y', source=source, line_width=2, color=color, name=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),
                    line_dash='dotted')
      
    
    #Add tooltip on hover:
    p.add_tools(HoverTool(tooltips=[
        ('Station Code',station_info_dict['station_code']),
        ('Latitude',station_info_dict['station_lat']),
        ('Longitude',station_info_dict['station_lon']),
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        (tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),'@y{0.f}'),
        ('St dev', '@w{0.f}'),
        ('NbPoints', '@z'),
        ('Flag', '@o')
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    


    #Set title attributes:
    p.title.align = 'center'
    p.title.text_font_size = '13pt'
    p.title.offset = 15

    #Set label font style:
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'
    
    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Add label to plot:
    p.add_layout(caption1, 'below')

    #return plot:
    return p

In [ ]:
def plot_icos_single_station_multiple_tracers_binary(df_list):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri Apr 07 10:00:00 2019
    Last Changed:     Fri Apr 07 10:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that creates an interactive Bokeh plot with ICOS Level-2 Atmospheric Data 
                      ('CO2', 'CO', 'CH4'). The plot has a sepparate y-axis for every tracer.
                      
    Input parameters: List of lists of station-info dictionaries, tracer-info dictionaries and
                      data dataframes with ICOS Level-2 Atmospheric Data
                      (var_name: "df_list", var_type: List of Pandas DataFrames)

    Output:           Bokeh Plot
    
    """
    
    #Import modules to create figure:
    import pandas as pd
    from bokeh.plotting import figure
    from bokeh.models import ColumnDataSource, HoverTool, Label, Legend, LinearAxis, Range1d, SingleIntervalTicker
    from datetime import datetime
    
 
    #Dictionaries for subscript/superscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

    #Create a figure object:
    p = figure(plot_width=900,
               plot_height=500,
               x_axis_label='Time (UTC)', 
               y_axis_label=  df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB)+' (' +
               df_list[0][1]['tracer_unit'].translate(SUP) + ')',
               x_axis_type='datetime',
               title = 'Tracer Observations ('+
               df_list[0][0]['station_name']+', '+
               df_list[0][0]['station_country']+', '+
               df_list[0][0]['station_sampling_height']+' m. a. g. l.)',
               tools='pan,box_zoom,wheel_zoom,reset, save')
    

    #Dictionary containing tracer colors:
    colors = {'co':'#ff7502',#'#bf812d','#d6604d','#CD6839','#f46d43','#d2691e', '#fc8d59', 
              'co2':'#543005',#'#48240A','#993404', 
              'ch4':'#e4c981'}#'#fee090'}'#dfc27d'}'#FFB90F'}'#fdcc8a'}
    
    #Create an empty list that will store the legend info:
    legend_it = []
    
    #Extract time and tracer values for every tracer category:
    x = pd.to_datetime(df_list[0][2]['TIMESTAMP'], unit='ms') 
    y = df_list[0][2][df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]

    #Create a circle and line glyph for the values of every emission category:
    r0 = p.circle(x, y, radius=.12, color=colors[df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()])
    r1 = p.line(x, y, line_width=1, color=colors[df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()],
                name=df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB))

    #Add the name and glyph info (i.e. colour and marker type) to the legend:
    legend_it.append((df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB), [r0,r1]))
    
    
    #If 2 tracers have been selected:
    if(len(df_list)==2):
        
        #Get the total min/max values for every tracer:
        tracer1_min = df_list[0][2][df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].min()
        tracer1_max = df_list[0][2][df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].max()
        tracer2_min = df_list[1][2][df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].min()
        tracer2_max = df_list[1][2][df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].max()
    
        #If two tracers are selected anf if one of the tracers is co2:
        if((len(df_list)==2) and (df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','')=='CO2')):
            p.y_range, p.extra_y_ranges = set_yranges_2y(rounddown_20(tracer1_min),
                                                         roundup_20(tracer1_max), 
                                                         rounddown_100(tracer2_min), 
                                                         roundup_100(tracer2_max), 20.0, 100.0, 'Yaxis2')

            #Set primary y-axis ticker:
            ticker_1 = SingleIntervalTicker(interval= 20)

            #Add primary y-axis ticker to plot:
            p.yaxis.ticker = ticker_1

            #Set secondary y-axis ticker:
            ticker_2 = SingleIntervalTicker(interval=100)

            #Create 2nd y-axis: 
            bg_yaxis = LinearAxis(y_range_name="Yaxis2",
                                  axis_label=df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB)+' (' +
                                  df_list[1][1]['tracer_unit'].translate(SUP) + ')',
                                  ticker=ticker_2, axis_label_standoff = 15) 

            #Add secondary y-axis to plot:
            p.add_layout(bg_yaxis, 'right')

        else: 
            p.y_range, p.extra_y_ranges = set_yranges_2y(rounddown_100(tracer1_min),
                                                         roundup_100(tracer1_max),
                                                         rounddown_100(tracer2_min),
                                                         roundup_100(tracer2_max),
                                                         100.0, 100.0, 'Yaxis2')

            #Set primary y-axis ticker:
            ticker_1 = SingleIntervalTicker(interval= 100)

            #Add primary y-axis ticker to plot:
            p.yaxis.ticker = ticker_1

            #Set secondary y-axis ticker:
            ticker_2 = SingleIntervalTicker(interval=100)

            #Create 2nd y-axis: 
            yaxis_2 = LinearAxis(y_range_name="Yaxis2",
                                 axis_label=df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB)+' (' +
                                 df_list[1][1]['tracer_unit'].translate(SUP) + ')', 
                                 ticker=ticker_2, 
                                 axis_label_standoff = 15) 

            #Add secondary y-axis to plot:
            p.add_layout(yaxis_2, 'right')
            
            
            
            
            
        #Set the text color of the yaxis for both y-axes:
        p.yaxis[0].axis_label_text_color = colors[df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]
        p.yaxis[1].axis_label_text_color = colors[df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]
    
    
    
    
    
        #Extract time and tracer values for every tracer category:
        x2i = pd.to_datetime(df_list[1][2]['TIMESTAMP'], unit='ms') 
        y2i = df_list[1][2][df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]

        #Create a circle and line glyph for the values of every emission category:
        r2 = p.circle(x2i, y2i, radius=.12, color=colors[df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()],
                      y_range_name="Yaxis2")
        r3 = p.line(x2i, y2i, line_width=1, color=colors[df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()],
                    name=df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB), y_range_name="Yaxis2")

        #Add the name and glyph info (i.e. colour and marker type) to the legend:
        legend_it.append((df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB), [r2,r3]))

        
 
        
        
    #If three tracers have been selected:    
    if(len(df_list)==3):
        
        #Get the total min/max values for every tracer:
        tracer1_min = df_list[0][2][df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].min()
        tracer1_max = df_list[0][2][df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].max()
        tracer2_min = df_list[1][2][df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].min()
        tracer2_max = df_list[1][2][df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].max()
        tracer3_min = df_list[2][2][df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].min()
        tracer3_max = df_list[2][2][df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].max()
    
        #Get the ranges for every y-axis:
        p.y_range, p.extra_y_ranges['Yaxis2'], p.extra_y_ranges['Yaxis3']= set_yranges_3y(rounddown_20(tracer1_min),
                                                                                          roundup_20(tracer1_max), 
                                                                                          rounddown_100(tracer2_min), 
                                                                                          roundup_100(tracer2_max),
                                                                                          rounddown_100(tracer3_min), 
                                                                                          roundup_100(tracer3_max),
                                                                                          20.0, 100.0, 100.0)

        #Set primary y-axis ticker:
        ticker_1 = SingleIntervalTicker(interval= 20)

        #Add primary y-axis ticker to plot:
        p.yaxis.ticker = ticker_1

        #Set secondary y-axis ticker:
        ticker_2 = SingleIntervalTicker(interval=100)

        #Create 2nd y-axis: 
        yaxis_2 = LinearAxis(y_range_name="Yaxis2",
                             axis_label=df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB)+' (' +
                             df_list[1][1]['tracer_unit'].translate(SUP) + ')',
                             ticker=ticker_2, 
                             axis_label_standoff = 15) 
        
        #Create 3rd y-axis:
        yaxis_3 = LinearAxis(y_range_name="Yaxis3", 
                             axis_label=df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB)+' (' +
                             df_list[2][1]['tracer_unit'].translate(SUP) + ')',
                             ticker=ticker_2,
                             axis_label_standoff = 15) 
        
        #Add secondary y-axis to plot:
        p.add_layout(yaxis_2, 'right') 
        
        #Add third y-axis to plot:
        p.add_layout(yaxis_3, 'right') 
        
        df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower
        
        #Set yaxis tick-label color for all 3 y-axes:
        p.yaxis[0].major_label_text_color = colors[df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]
        p.yaxis[1].major_label_text_color = colors[df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]
        p.yaxis[2].major_label_text_color = colors[df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]
        
        #Set the text color of the yaxis for all 3 y-axes:
        p.yaxis[0].axis_label_text_color = colors[df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]
        p.yaxis[1].axis_label_text_color = colors[df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]
        p.yaxis[2].axis_label_text_color = colors[df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]
        
        
        
        
        
        
        
        

        #Extract time and tracer values for every tracer category:
        x2 = pd.to_datetime(df_list[1][2]['TIMESTAMP'], unit='ms') 
        y2 = df_list[1][2][df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]

        #Create a circle and line glyph for the values of every emission category:
        r2 = p.circle(x2, y2, radius=.12,
                      color=colors[df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()],
                      y_range_name="Yaxis2")
        r3 = p.line(x2, y2, line_width=1,
                    color=colors[df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()],
                    name=df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB), y_range_name="Yaxis2")

        #Add the name and glyph info (i.e. colour and marker type) to the legend:
        legend_it.append((df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB), [r2,r3]))
        
        #Extract time and tracer values for every tracer category:
        x3 = pd.to_datetime(df_list[2][2]['TIMESTAMP'], unit='ms') 
        y3 = df_list[2][2][df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]

        #Create a circle and line glyph for the values of every emission category:
        r4 = p.circle(x3, y3, radius=.12,
                      color=colors[df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()],
                      y_range_name="Yaxis3")
        r5 = p.line(x3, y3, line_width=1,
                    color=colors[df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()],
                    name=df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB), y_range_name="Yaxis3")

        #Add the name and glyph info (i.e. colour and marker type) to the legend:
        legend_it.append((df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB), [r4,r5]))
        
        
        
        
        
    #Add tooltip on hover:
    p.add_tools(HoverTool(tooltips=[
        ('Tracer type','$name'),
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        ('Tracer value','@y{0.f}'),
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    

    #Create legend:
    legend = Legend(items=legend_it, location= 'bottom_center')
    legend.orientation = 'horizontal'
    legend.click_policy='hide'
    legend.spacing = 10 #sets the distance between legend entries

    #Set title attributes:
    p.title.align = 'center'
    p.title.text_font_size = '13pt'
    p.title.offset = 15

    #Set axis label font style:
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'
    
    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Add label to plot:
    p.add_layout(caption1, 'below')
    
    #Add legend to figure:
    p.add_layout(legend, 'below')

    #return plot:
    return p


In [ ]:
def plot_icos_single_tracer_multiple_stations_binary(df_list):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes as input a list of sublists with dataframes, where every sublist
                      contains a metadata and a data dataframe for a tracer measured at a specific station. 
                      The function creates an interactive Bokeh plot with the contents of the dataframes and
                      returns a Bokeh Figure (plot).
                      
                      
    Input parameters: List of sublists with ICOS Level-2 Atmospheric data and metadata dataframes
                     (var_name: 'df_list', var_type: List)

    Output:           Bokeh Plot
    
    """
    
    #Import modules to create figure:
    import pandas as pd
    from bokeh.plotting import figure
    from bokeh.models import ColumnDataSource, HoverTool, Label, Legend
    from datetime import datetime

    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

    

    #Create a figure object:
    p = figure(plot_width=900,
               plot_height=500,
               x_axis_label='Time (UTC)', 
               y_axis_label=df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)', '').translate(SUB)+
               ' (' + df_list[0][1]['tracer_unit'].translate(SUP) + ')',
               x_axis_type='datetime',
               title = df_list[0][1]['tracer_info'].translate(SUB),
               tools='pan,box_zoom,wheel_zoom,reset,save')

    #Get colormap:
    colormap = get_colormap(len(df_list))
    
    #Create an empty list that will store the legend info:
    legend_it = []

    #Create a glyph with a different colour for every tracer:
    for item, color in zip(df_list, colormap):

        #Extract time and tracer values for every tracer category:
        x = pd.to_datetime(item[2]['TIMESTAMP'], unit='ms') 
        y = item[2][item[1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()] 
        

        #Create a circle and line glyph for the values of every emission category:
        r0 = p.circle(x, y, radius=.12, color=color)
        r1 = p.line(x, y, line_width=1, color=color, name=item[0]['station_code']+' ('+
                    item[0]['station_sampling_height']+')')

        #Add the name and glyph info (i.e. colour and marker type) to the legend:
        legend_it.append((item[0]['station_code']+' ('+item[0]['station_sampling_height']+')', [r0,r1]))



    #Add tooltip on hover:
    p.add_tools(HoverTool(tooltips=[
        ('Station','$name'),
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        (item[1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),'@y{0.f}'),
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    

    #Create legend:
    legend = Legend(items=legend_it, location= 'bottom_center')
    legend.orientation = 'horizontal'
    legend.click_policy='hide'
    legend.spacing = 10 #sets the distance between legend entries

    #Set title attributes:
    p.title.align = 'center'
    p.title.text_font_size = '13pt'
    p.title.offset = 15

    #Set axis label font style:
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'
    
    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Add label to plot:
    p.add_layout(caption1, 'below')
    
    #Add legend to figure:
    p.add_layout(legend, 'below')

    #return plot:
    return p
    


In [ ]:
def plot_icos_focus_binary(df_data, station_info_dict, tracer_info_dict, tracer, color):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes as input a metadata dataframe, a data dataframe, a string stating
                      the tracer and a string with the color for the plot of datafor a specific ICOS station. 
                      The function creates two interactive Bokeh plots with the contents of the dataframes and
                      returns two Bokeh Figures (plots).
                      
                      
    Input parameters: 1. ICOS Level-2 tracer Atmospheric Data Dataframe
                         (var_name: 'df_data', var_type: Pandas DataFrame)
                      2. Dictionary with ICOS station information 
                         (var_name: 'station_info_dict', var_type: Dictionary)
                      3. Dictionary with tracer information 
                         (var_name: 'tracer_info_dict', var_type: Dictionary)
                      4. Tracer/gas - e.g. 'co2'
                         (var_name: "tracer", var_type: String)
                      5. Plot Color
                         (var_name: "color", var_type: String)

    Output:           Bokeh Plot
    
    """

    #Import modules to create figure:
    #from bokeh.palettes import Category10
    import pandas as pd
    from bokeh.layouts import row, column
    from bokeh.models import ColumnDataSource, HoverTool, Label, Legend, CustomJS, Rect
    from bokeh.plotting import figure, show
    from datetime import datetime
    from bokeh.io import push_notebook, output_notebook


    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

    #Define Datasets:
    x = pd.to_datetime(df_data['TIMESTAMP'], unit='ms')
    y = df_data[tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].values 

    #Create a ColumnDataSource object:
    source = ColumnDataSource({'x': [], 'y': [], 'width': [], 'height': []})

    #Javascript code defining the new x-range, y-range of the plot, based on the area selected by the user:
    jscode="""
        var data = source.data;
        var start = cb_obj.start;
        var end = cb_obj.end;
        data['%s'] = [start + (end - start) / 2];
        data['%s'] = [end - start];
        source.change.emit();
    """

    #Create a figure object:
    p1 = figure(plot_width=900,
                plot_height=400,
                x_axis_label='Time (UTC)',
                y_axis_label=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB) + ' (' +
                tracer_info_dict['tracer_unit'].translate(SUP) + ')',
                x_axis_type='datetime',
                title = tracer_info_dict['tracer_info'].translate(SUB)+'   '+
                station_info_dict['station_name']+', '+
                station_info_dict['station_country'][0]+', '+
                station_info_dict['station_sampling_height']+' m. a. g. .l.',
                tools='pan,box_zoom,wheel_zoom,undo,redo,reset,save')

    #Create glyphs:
    p1.circle(x,y, radius=.02, color=color)# ,legend=df_metadata.loc['STATION CODE'].values[0])
    p1.line(x,y, line_width=1, color=color)# ,legend=df_metadata.loc['STATION CODE'].values[0])

    #Add tooltip on hover:
    p1.add_tools(HoverTool(tooltips=[
        ('Station Code',station_info_dict['station_code']),
        ('Latitude',station_info_dict['station_lat']),
        ('Longitude',station_info_dict['station_lon']),
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        (tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),'@y{0.f}'),
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    

    #Set title attributes:
    p1.title.align = 'center'
    p1.title.text_font_size = '13pt'
    p1.title.offset = 15

    #Set label font style:
    p1.xaxis.axis_label_text_font_style = 'normal'
    p1.yaxis.axis_label_text_font_style = 'normal'
    p1.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p1.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'
    
    #Change the plot's x-range, y-range based on the selected area (javascript callback):   
    xcb = CustomJS(args=dict(source=source), code=jscode % ('x', 'width'))
    ycb = CustomJS(args=dict(source=source), code=jscode % ('y', 'height'))
    p1.x_range.js_on_change('start', xcb)
    p1.x_range.js_on_change('end', xcb)
    p1.y_range.js_on_change('start', ycb)
    p1.y_range.js_on_change('end', ycb)
    
    #Deactivate hover-tool, which is by default active:
    p1.toolbar.active_inspect = None

    #Add label to plot:
    p1.add_layout(caption1, 'below')

    
    ############################################### CODE FOR 2nd PLOT ###################################################


    #Create a figure object:
    p2 = figure(plot_width=900,
               plot_height=400,
               x_axis_label='Time (UTC)', 
               y_axis_label=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB) + 
               ' (' +tracer_info_dict['tracer_unit'].translate(SUP) + ')',
               x_axis_type='datetime',
               title = tracer_info_dict['tracer_info'].translate(SUB)+'   '+
               station_info_dict['station_name']+', '+
               station_info_dict['station_country']+', '+
               station_info_dict['station_sampling_height']+' m. a. g. l.',
               tools='save')

    #Create glyphs:
    p2.circle(x, y, radius=.02, color=color)# ,legend=df_metadata.loc['STATION CODE'].values[0])
    p2.line(x, y, line_width=1, color=color)# ,legend=df_metadata.loc['STATION CODE'].values[0])

    #Add tooltip on hover:
    p2.add_tools(HoverTool(tooltips=[
        ('Station Code',station_info_dict['station_code']),
        ('Latitude',station_info_dict['station_lat']),
        ('Longitude',station_info_dict['station_lon']),
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        (tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),'@y{0.f}')
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    



    #Set title attributes:
    p2.title.align = 'center'
    p2.title.text_font_size = '13pt'
    p2.title.offset = 15

    #Set label font style:
    p2.xaxis.axis_label_text_font_style = 'normal'
    p2.yaxis.axis_label_text_font_style = 'normal'
    p2.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p2.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units


    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'

    #Add label to plot:
    p2.add_layout(caption1, 'below')
    
    #Add a rectangle with 
    rect = Rect(x='x', y='y', width='width', height='height', fill_alpha=0.1, line_color='black', fill_color='black')
    p2.add_glyph(source, rect)
    
    #Deactivate hover-tool, which is by default active:
    p2.toolbar.active_inspect = None

    #Return plots:
    return p1, p2

In [ ]:
def plot_icos_single_station_smoothing_binary(df_data, station_info_dict, tracer_info_dict, tracer, color):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes as input a metadata dataframe, a data dataframe, a string stating
                      the tracer and a string with the color for the plot of datafor a specific ICOS station. 
                      The function creates an interactive Bokeh plot with the contents of the dataframes and
                      returns a Bokeh Figure (plot).
                      
                      
    Input parameters: 1. ICOS Level-2 tracer Atmospheric Data Dataframe
                         (var_name: 'df_data', var_type: Pandas DataFrame)
                      2. Dictionary with ICOS Station info
                         (var_name: 'station_info_dict', var_type: Dictionary)
                      3. Dictionary with tracer info
                         (var_name: 'tracer_info_dict', var_type: Dictionary)
                      4. Tracer/gas - e.g. 'co2'
                         (var_name: "tracer", var_type: String)
                      5. Plot Color
                         (var_name: "color", var_type: String)

    Output:           Bokeh Plot
    
    """
    
    #Import modules to create figure:
    import pandas as pd
    from bokeh.plotting import figure
    from bokeh.models import ColumnDataSource, HoverTool, Label, Legend
    from datetime import datetime

    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

    #Define Datasets:
    x = df_data.index.values
    y1 = df_data['rolling_mean']#.astype({tracer: np.float32})[tracer] 
    y2 = df_data[tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]

    #Create a figure object:
    p = figure(plot_width=900,
               plot_height=400,
               x_axis_label='Time (UTC)', 
               y_axis_label=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB) + ' (' +
               tracer_info_dict['tracer_unit'].translate(SUP) + ')',
               x_axis_type='datetime',
               title = tracer_info_dict['tracer_info'].translate(SUB)+'   '+
               station_info_dict['station_name']+', '+
               station_info_dict['station_country']+', '+
               station_info_dict['station_sampling_height']+' m. a. g. l.',
               tools='pan,box_zoom,wheel_zoom,,undo,reset,save')

    #Create an empty list that will store the legend info:
    legend_it = []
    
    #Create glyphs:
    g0 = p.circle(x, y1, radius=.02, color=color)# ,legend=df_metadata.loc['STATION CODE'].values[0])
    g1 = p.line(x, y1, line_width=1.25, color=color, name='Smoothed Line')
    
    g2 = p.circle(x, y2, radius=.02, color=color)# ,legend=df_metadata.loc['STATION CODE'].values[0])
    g3 = p.line(x, y2, line_width=1, line_dash='dotted', line_alpha=0.5, color=color, name='Original Observations')
    
    #Add the name and glyph info (i.e. colour and marker type) to the legend:
    legend_it.append(('Smoothed '+station_info_dict['station_code']+' ('+
                      station_info_dict['station_sampling_height']+')', [g1]))
    
    #Add the name and glyph info (i.e. colour and marker type) to the legend:
    legend_it.append(('Original ' + station_info_dict['station_code']+' ('+
                      station_info_dict['station_sampling_height']+')', [g3]))

    
    #Add tooltip on hover:
    p.add_tools(HoverTool(tooltips=[
        ('Type','$name'),
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        (tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),'@y{0.f}'),
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    

    #Create legend:
    legend = Legend(items=legend_it, location= 'bottom_center')
    legend.orientation = 'horizontal'
    legend.click_policy='hide'
    legend.spacing = 10 #sets the distance between legend entries

    #Set title attributes:
    p.title.align = 'center'
    p.title.text_font_size = '13pt'
    p.title.offset = 15

    #Set label font style:
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'
    
    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Add label to plot:
    p.add_layout(caption1, 'below')
    
    #Add legend to figure:
    p.add_layout(legend, 'below')

    #return plot:
    return p

In [ ]:
def plot_icos_single_station_L1_L2_binary(data_df_list, station_info_dict, tracer_info_dict, color):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes as input a list of data dataframes, a dictionary with information for 
                      the ICOS station, a dictionary with tracer information and a string with the color for 
                      the plot line. This implementation is for one selected ICOS station and one selected tracer. 
                      The function creates an interactive Bokeh plot with the contents of the dataframes and
                      returns a Bokeh Figure (plot).
                      
                      
    Input parameters: 1. ICOS Level-2 tracer Atmospheric Data Dataframe
                         (var_name: 'data_df_list', var_type: List of dataframes)
                      2. Dictionary with info for ICOS stations
                         (var_name: 'station_info_dict', var_type: Dictionary)
                      3. Dictionary with tracer info (i.e. tracer name, tracer unit)
                         (var_name: "tracer_info_dict", var_type: Dictionary)
                      4. Selected Color for plot line
                         (var_name: "color", var_type: String)

    Output:           Bokeh Plot
    
    """
    
    #Import modules to create figure:
    from bokeh.plotting import figure
    from bokeh.models import ColumnDataSource, HoverTool, Label, Legend
    from datetime import datetime

    
    #Dictionaries for subscript/superscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

    #Create a figure object:
    p = figure(plot_width=900,
               plot_height=500,
               x_axis_label='Time (UTC)', 
               y_axis_label=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)', '').translate(SUB)+
               ' (' +tracer_info_dict['tracer_unit'].translate(SUP) + ')',
               x_axis_type='datetime',
               title = tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)', '').translate(SUB)+
               ' - Continuous air ( '+
               station_info_dict['station_name']+', '+#.encode('latin1').decode('utf8')+', '+
               station_info_dict['station_country']+', '+
               station_info_dict['station_sampling_height']+' m. a. g. l.)' ,
               tools='pan,box_zoom,wheel_zoom,reset,save')
    
    
    #Create an empty list that will store the legend info:
    legend_it = []

   

    #Extract time and tracer values for every data level:
    x1 = data_df_list[0].index.values
    y1 = data_df_list[0][tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)', '').lower()].values
    x2 = data_df_list[1].index.values
    y2 = data_df_list[1][tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)', '').lower()].values

    #Create a circle and line glyph for the values of every emission category:
    r0 = p.circle(x1, y1, radius=.12, color=color, alpha=0.5)
    r1 = p.line(x1, y1, line_width=2, line_dash='dotted', line_alpha=0.5, color=color, name='L1, '+station_info_dict['station_code']+' ('+station_info_dict['station_sampling_height']+')')
    r2 = p.circle(x2, y2, radius=.12, color=color)
    r3 = p.line(x2, y2, line_width=1, color=color, name='L2, '+station_info_dict['station_code']+' ('+station_info_dict['station_sampling_height']+')')
    
    #Add the name and glyph info (i.e. colour and marker type) to the legend:
    legend_it.append(('Level 1 - '+station_info_dict['station_code']+' ('+station_info_dict['station_sampling_height']+')', [r1]))
    
    #Add the name and glyph info (i.e. colour and marker type) to the legend:
    legend_it.append(('Level 2 - '+station_info_dict['station_code']+' ('+station_info_dict['station_sampling_height']+')', [r3]))



    #Add tooltip on hover:
    p.add_tools(HoverTool(tooltips=[
        ('Station','$name'),
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        (tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),'@y{0.f}'),
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    

    #Create legend:
    legend = Legend(items=legend_it, location= 'bottom_center')
    legend.orientation = 'horizontal'
    legend.click_policy='hide'
    legend.spacing = 10 #sets the distance between legend entries

    #Set title attributes:
    p.title.align = 'center'
    p.title.text_font_size = '13pt'
    p.title.offset = 15

    #Set axis label font style:
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'
    
    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Add label to plot:
    p.add_layout(caption1, 'below')
    
    #Add legend to figure:
    p.add_layout(legend, 'below')

    #return plot:
    return p
    

<a id='widget_funcs'></a>
<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>
<br>
<br>

## 8. Widget functions
This part includes functions that create widget-forms and help functions to these widget-forms. In Python interactive elements like dropdown lists, ckeckboxes, buttons, etc. are called widgets.

In [ ]:
def create_station_labels(lookup_df,st_name_col, st_id_col, st_sh_col):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Mon Apr 01 10:26:00 2019
    Last Changed:     Tue Nov 17 09:30:00 2020
    Version:          1.1.0
    Author(s):        Karolina Pantazatou
    
    Description:      Return a list of tuples. Every tuple contains the label for a specific station and a
                      list with the station's 3-character code and sampling height. The first item of every tuple
                      is used as a label in the station dropdown list (i.e. Select or Multi-select widget).
                      
    Input parameters: 1. lookup table (pandas dataframe)
                      2. name of column in df with station name info (var_name: "st_name_col", var_type: String)
                      3. name of column in df with station id info (var_name: "st_id_col", var_type: String)
                      4. name of column with station sampling height info (var_name: "st_sh_col", var_type: String)
    
    Output:           List of tuples (e.g. [('Gartow (alt. 30.0)', ['GAT', '30.0']),...]
                      tuple: 
                            1. Constructed Station Label (var_type: String)
                            2. List of two items
                                i. Station 3-character Code (var_type: String)
                                ii. Station Sampling Height (var_type: String)

    """
    
    #Filter the dataframe and get a dataframe of unique combinations of station-names and 
    #corresponding sampling heights:
    df = lookup_df.filter([st_name_col, st_sh_col, st_id_col]).drop_duplicates(subset=[st_name_col,
                                                                                       st_sh_col,
                                                                                       st_id_col])

    #Get a list of tuples for every station that has provided tracer-data:
    #Every tuple is constructed like: ('Gartow (alt. 30.0)', ['GAT', '30.0'])
    station_labels = [(df[st_name_col].iloc[i]+ " (alt. " + df[st_sh_col].iloc[i] + ")",
                       [df[st_id_col].iloc[i], df[st_sh_col].iloc[i]]) for i in range(len(df))]
    
    #Return list:
    return station_labels
    

In [ ]:
def create_prod_label(lbl_ls, prod_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue Nov 01 09:30:00 2020
    Last Changed:     Tue Nov 17 09:30:00 2020
    Version:          0.0.1
    Author(s):        Karolina Pantazatou
    
    Description:      Return a list of tuples. Every tuple contains the label for a specific data product
                      and a list with the ICOS data product's official term as well as a shortened version
                      of it (that serves as a label). The first item of every tuple is used as a label in
                      the data product dropdown list (i.e. Select or Multi-select widget).
                      
    Input parameters: 1. lookup table (pandas dataframe)
                      2. name of column in df with station name info (var_name: "st_name_col", var_type: String)
                      3. name of column in df with station id info (var_name: "st_id_col", var_type: String)
                      4. name of column with station sampling height info (var_name: "st_sh_col", var_type: String)
    
    Output:           List of tuples (e.g. [('CH4', ['ICOS ATC CH4 Release', 'CH4']),...]
                      tuple: 
                            1. Constructed Data Product Label (var_type: String)
                            2. List of two items
                                i. Official ICOS Data Product term (var_type: String)
                                ii. Data Product Label (var_type: String)

    """
    
    #Check if input parameters are lists & have the same length:
    if((isinstance(lbl_ls, list)) & (isinstance(prod_ls, list)) & (len(lbl_ls)==len(prod_ls))):
        
        #Return list of tuples with label- and data prod-info:
        return [(lbl_ls[i], [prod_ls[i], lbl_ls[i]]) for i in range(len(lbl_ls))]
    
    #If input parameters are not lists or are have the same length:
    else:
        
        #Prompt error message:
        print('Input parameters must be lists of the same length')
        
        return False

In [ ]:
def create_widgets_exploring(st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri Apr 07 10:00:00 2019
    Last Changed:     Wed Nov 18 15:00:00 2020
    Version:          0.1.2
    Author(s):        Karolina Pantazatou
    
    Description:      Function that creates a set of widgets; a tracer dropdown list, a station dropdown list,
                      a basemap dropdown list, a colorpicker and a button, populates the dropdown lists with
                      values, captures the user's input and calls a function to update the contents of the
                      "exploring"-plot.
                      
    Input parameters: List of ICOS station objects
                      (var_name: 'st_obj_ls', var_type: List)

    Output:           Plot with Map or Warning Message
    
    """
    
    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

    #Create lookup dataframe:
    df_lookup = create_lookup_df_atc_L2(st_obj_ls)

    #Create widgets:
    tracer = Dropdown(options = create_prod_label(['CH4', 'CO', 'CO2'], sorted(df_lookup['specLabel'].unique().tolist())))
    station =  Dropdown(options = sorted(create_station_labels(df_lookup, 'stationName', 'stationId', 'samplingheight'),
                                         key = lambda x: (x[0].split(' (', 1)[0], float(x[1][1]))))
    basemap_wdgt = Dropdown(options = ['Imagery', 'OpenStreetMap'], description='Basemap')

    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Tracer, Station, basemap, color, Citation):

        #Get a list of data obect URLs that refer to the selected station and tracer:
        data_obj_url_ls =list(df_lookup.dobj.loc[(df_lookup['stationId']==Station[0])&
                                                 (df_lookup['samplingheight']==Station[1])&
                                                 (df_lookup['specLabel']==Tracer[0])])


        #If L2-data is available for the selected tracer and station:
        if(len(data_obj_url_ls)>0):


            #Call function to return plot for the selected station:
            p = update_icos_single_station_plot_binary(data_obj_url_ls, Station, Tracer[1].lower(), color, st_obj_ls)

            #Output should be in the notebook
            output_notebook()

            #Show plot:
            show(p, notebook_handle=True)

            #If the "citation" checkbox is checked:
            if(Citation):

                #Get a list with citation info for every ICOS Level-2 data object:
                cit_ls_L2 = [RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                             for dobj in data_obj_url_ls]


                #Print citation title:
                print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')

                #Loop through all citations:
                for cit in cit_ls_L2:

                    #Print data object citation:
                    printmd("<sub>"+cit+"</sub>")


            #Show map:
            plotmap(df_lookup, Station[0], basemap)

        #If no L2-data are available for the selected tracer and station:
        else:
            print("\033[0;31;1m "+'No '+Tracer[1].upper().translate(SUB)+' Level-2 data available for the selected station...'+"\033[0;31;0m\n\n")






    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracer=tracer,
                                 Station=station,
                                 basemap = basemap_wdgt,
                                 color=ColorPicker(concise=False,
                                                   description='Pick a color',
                                                   value='#3973ac',
                                                   disabled=False),
                                 Citation=Checkbox(value=True, description='Citation', disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '430px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 2px'
    interact_c.widget.children[1].layout.width = '430px'
    interact_c.widget.children[2].layout.width = '430px'
    interact_c.widget.children[3].layout.width = '430px'
    interact_c.widget.children[4].layout.width = '430px'
    interact_c.widget.children[5].description = 'Update Plot'
    interact_c.widget.children[5].button_style = 'danger'
    interact_c.widget.children[5].style.button_color = '#3973ac'
    interact_c.widget.children[5].layout.margin = '10px 10px 40px 180px' # top/right/bottom/left

In [ ]:
def create_widgets_exploring_multiple_tracers(st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Wed Nov 18 16:30:00 2020
    Version:          0.1.3
    Author(s):        Karolina
    
    Description:      Function that creates a set of widgets; a tracer multiselect dropdown list, a
                      station dropdown list and a button. The function populates the dropdown lists
                      with values and outputs the result.
                      
                      
    Input parameters: List of ICOS station objects
                      (var_name: 'st_obj_ls', var_type: List)

    Output:           Python Widgets
    
    """

    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

    #Create lookup dataframe:
    df_lookup = create_lookup_df_atc_L2(st_obj_ls)

    #Create widgets:
    tracer = SelectMultiple(options = create_prod_label(['CH4', 'CO', 'CO2'], sorted(df_lookup['specLabel'].unique().tolist())))
    station =  Dropdown(options = sorted(create_station_labels(df_lookup, 'stationName', 'stationId', 'samplingheight'),
                                         key = lambda x: (x[0].split(' (', 1)[0], float(x[1][1]))))
    

    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Tracer, Station, Citation):
      
        #Get a list of Level-2 data object URLs, tracers, station-IDs and station sampling heights
        #that refer to the selected station and tracer/s:
        selection_dobj_url_list = [[df_lookup.dobj.loc[(df_lookup['stationId']==Station[0]) &
                                                       (df_lookup['samplingheight']==Station[1]) & 
                                                       (df_lookup['specLabel']==tracer[0])].values,
                                    tracer[1].lower(),
                                    Station[0],
                                    Station[1]]
                                   for tracer in Tracer
                                   if len(df_lookup.dobj.loc[(df_lookup['stationId']==Station[0]) &
                                                             (df_lookup['samplingheight']==Station[1]) &
                                                             (df_lookup['specLabel']==tracer[0])].values)>0]

        
        #If Level-2 data are available for the selected tracer and station:
        if(len(selection_dobj_url_list)>0):
            
            #Get a list of lists, where every sublist contains
            #a data-object-id, a tracer-string (e.g. 'ch4'), a station-ID string (e.g. 'GAT')
            #and the station sampling height string (e.g. '30.0'):
            selection_list = [[selection_dobj_url_list[item][0][0],
                               selection_dobj_url_list[item][1],
                               selection_dobj_url_list[item][2],
                               selection_dobj_url_list[item][3]]
                              for item in range(len(selection_dobj_url_list))]
            

            #Call function to return plot for the selected station:
            update_exploring_multiple_tracers_binary(selection_list, st_obj_ls)
            
            #If the "citation" checkbox is checked:
            if(Citation):
                
                #Get a list with citation info for every ICOS Level-2 data object:
                cit_ls_L2 = [RunSparql(sparql_query=sparqls.get_icos_citation(dobj[0][0]), output_format='pandas').run().cit.iloc[0]
                             for dobj in selection_dobj_url_list]
                    
                #Print citation title:
                print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                    
                #Loop through all citations:
                for cit in cit_ls_L2:
                    
                    #Print data object citation:
                    printmd("<sub>"+cit+"</sub>")
        
        #If no L2-data is available for the selected tracer and station:
        else:
            print("\033[0;31;1m "+'No Level-2 data available for the selected tracer and/or station/s.'+"\033[0;31;0m\n\n")



    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracer=tracer,
                                 Station=station,
                                 Citation=Checkbox(value=True, description='Citation', disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '420px'
    interact_c.widget.children[0].layout.height = '60px'
    interact_c.widget.children[1].layout.width = '420px'
    interact_c.widget.children[2].layout.width = '420px'
    interact_c.widget.children[3].description = 'Update Plot'
    interact_c.widget.children[3].button_style = 'danger'
    interact_c.widget.children[3].style.button_color = '#3973ac'
    #interact_c.widget.children[3].layout.width = '300px'
    interact_c.widget.children[3].layout.margin = '10px 10px 20px 180px' # top/right/bottom/left
    


In [ ]:
def create_widgets_exploring_multiple_stations(st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Wed Nov 18 17:00:00 2020
    Version:          0.1.3
    Author(s):        Karolina Pantazatou
    
    Description:      Function that creates a set of widgets; a station dropdown list
                      (it is possible to select more than one stations),
                      a tracer dropdown list, a color-picker and a button. The function
                      populates the dropdown lists with values and outputs an interactive
                      plot. The plot includes measurements for the selected tracer from 
                      all selected stations
                                      
    Input parameters: List of ICOS station objects
                      (var_name: 'st_obj_ls', var_type: List)

    Output:           Python Widgets and Plot/Error Message
    
    """
    
    #Create lookup dataframe:
    df_lookup = create_lookup_df_atc_L2(st_obj_ls)

    #Create widgets:
    tracer = Dropdown(options = create_prod_label(['CH4', 'CO', 'CO2'], sorted(df_lookup['specLabel'].unique().tolist())))
    station =  SelectMultiple(options = sorted(create_station_labels(df_lookup, 'stationName', 'stationId', 'samplingheight'),
                                         key = lambda x: (x[0].split(' (', 1)[0], float(x[1][1]))))
    

    
    
    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Tracer, Station, Citation):
        
        #Get a list of sublists, where every sublist contains the following:
        #1. ICOS Level-2 data object URL
        #2. Tracer/Gas (e.g. 'co2')
        #3. ICOS Station ID (3-character code)
        #4. ICOS Station Sampling Height
        #that refer to the selected station(s) and tracer:
        selection_dobj_url_list = [[df_lookup.dobj.loc[(df_lookup['stationId']==station[0]) &
                                                       (df_lookup['samplingheight']==station[1]) &
                                                       (df_lookup['specLabel']==Tracer[0])].values,
                                    Tracer[1].lower(),
                                    station[0],
                                    station[1]]
                                   for station in Station
                                   if len(df_lookup.dobj.loc[(df_lookup['stationId']==station[0]) &
                                                              (df_lookup['samplingheight']==station[1]) &
                                                              (df_lookup['specLabel']==Tracer[0])].values)>0]
        
        
        #If Level-2 data are available for the selected tracer and station(s):
        if(len(selection_dobj_url_list)>0):

            #Get a list of lists, where every sublist contains a data-object-ID, the selected tracer,
            #the station code and the station sampling height.
            #E.g. ['U4VYazHdmZwzr7DxUowMtUu-', 'co2', 'GAT', '30.0']:
            selection_list = [[selection_dobj_url_list[i][0][j],
                               selection_dobj_url_list[i][1],
                               selection_dobj_url_list[i][2],
                               selection_dobj_url_list[i][3]]
                              for i in range(len(selection_dobj_url_list))
                              for j in range(len(selection_dobj_url_list[i][0]))]
            
            ####
            #ICOS Atmospheric Level-2 Data for a given station, a given tracer and  
            #a given sampling height can be stored in two different files in cases
            #where the measuring instrument has changed.
            #The following code controls for such occurances and merges the data
            #if necessary.
            ####
            
            #Get a list of tuples (e.g. ('co2', 'HPB', '50.0')) with unique occurances of
            #"station code" - "station sampling height" - "tracer" triplets:
            station_unique_ls = list(set([(item[1], item[2], item[3]) for item in selection_list]))
            
            #Group selection_list items refering to tracer-data from the same station and
            #sampling height to lists:  
            station_dobj_ls = [[item for item in selection_list
                                if((item[1]==station_id[0]) & (item[2]==station_id[1]) & (item[3]==station_id[2]))]
                               for station_id in station_unique_ls]
            
            #Get plot displaying tracer-values for the selected station/s:
            update_exploring_multiple_stations_binary(station_dobj_ls, st_obj_ls)
            
            #If the "citation" checkbox is checked:
            if(Citation):
                
                #Get a list with citation info for every ICOS Level-2 data object:
                cit_ls_L2 = [RunSparql(sparql_query=sparqls.get_icos_citation(dobj[0][0]), output_format='pandas').run().cit.iloc[0]
                             for dobj in selection_dobj_url_list]
                
                    
                #Print citation title:
                print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                    
                #Loop through all citations:
                for cit in cit_ls_L2:
                    
                    #Print data object citation:
                    printmd("<sub>"+cit+"</sub>")
        
        #If no L2-data is available for the selected tracer and station:
        else:
            print("\033[0;31;1m "+'No Level-2 data available for the selected tracer and/or station/s.'+"\033[0;31;0m\n\n")
                


    
    
    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracer=tracer,
                                 Station=station,
                                 Citation=Checkbox(value=True, description='Citation', disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '420px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 2px'
    interact_c.widget.children[1].layout.width = '420px'
    interact_c.widget.children[1].layout.height = '115px'
    interact_c.widget.children[2].layout.width = '420px'
    interact_c.widget.children[3].description = 'Update Plot'
    interact_c.widget.children[3].button_style = 'danger'
    interact_c.widget.children[3].style.button_color = '#3973ac'
    interact_c.widget.children[3].layout.margin = '10px 10px 40px 180px' # top/right/bottom/left

In [ ]:
def create_widgets_focusing(st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Wed Nov 18 18:00:00 2020
    Version:          0.1.3
    Author(s):        Karolina Pantazatou
    
    Description:      Function that creates a set of widgets; a station dropdown list,
                      a tracer dropdown list, a color-picker and a button. The function
                      populates the dropdown lists with values and outputs the result.
                                      
    Input parameters: List of ICOS station objects
                      (var_name: 'st_obj_ls', var_type: List)

    Output:           Python Widgets
    
    """
    
    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

    #Create lookup dataframe:
    df_lookup = create_lookup_df_atc_L2(st_obj_ls)

    #Create widgets:
    tracer = Dropdown(options = create_prod_label(['CH4', 'CO', 'CO2'], sorted(df_lookup['specLabel'].unique().tolist())))
    station =  Dropdown(options = sorted(create_station_labels(df_lookup, 'stationName', 'stationId', 'samplingheight'),
                                         key = lambda x: (x[0].split(' (', 1)[0], float(x[1][1]))))
    

    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Tracer, Station, color, Citation):
       
        #Get a list of data object URLs that refer to the selected station and tracer:
        data_obj_url_ls = list(df_lookup.dobj.loc[(df_lookup['stationId']==Station[0])&
                                                  (df_lookup['samplingheight']==Station[1])&
                                                  (df_lookup['specLabel']==Tracer[0])])


        #If L2-data is available for the selected tracer and station:
        if(len(data_obj_url_ls)>0):
            
            #Call function to return plot and map for the selected station:
            update_focus_plot_binary(data_obj_url_ls, Station, Tracer[1].lower(), color, st_obj_ls)
            
            #If the "citation" checkbox is checked:
            if(Citation):
                
                #Get a list with citation info for every ICOS Level-2 data object:
                cit_ls_L2 = [RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                             for dobj in data_obj_url_ls]
                    
                #Print citation title:
                print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                    
                #Loop through all citations:
                for cit in cit_ls_L2:
                    
                    #Print data object citation:
                    printmd("<sub>"+cit+"</sub>")
        
        #If no L2-data are available for the selected tracer and station:
        else:
            print("\033[0;31;1m "+'No '+Tracer[1].upper().translate(SUB)+' Level-2 data available for the selected station'+"\033[0;31;0m\n\n")
                

           
    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracer=tracer,
                                 Station=station,
                                 color=ColorPicker(concise=False,
                                                   description='Pick a color',
                                                   value='#3973ac',
                                                   disabled=False),
                                 Citation=Checkbox(value=True, description='Citation', disabled=False))
    
    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '420px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 2px'
    interact_c.widget.children[1].layout.width = '420px'
    interact_c.widget.children[2].layout.width = '420px'
    interact_c.widget.children[3].layout.width = '420px'
    interact_c.widget.children[4].description = 'Update Plot'
    interact_c.widget.children[4].button_style = 'danger'
    interact_c.widget.children[4].style.button_color = '#3973ac'
    interact_c.widget.children[4].layout.margin = '10px 10px 40px 180px' # top/right/bottom/left

In [ ]:
def create_widgets_basic_stat(st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Wed Nov 18 18:30:00 2020
    Version:          0.1.3
    Author(s):        Karolina Pantazatou
    
    Description:      Function that creates a set of widgets; a station multiselection dropdown list, a
                      tracer dropdown list and a button. The function populates the dropdown lists with
                      values and outputs the result.
                      
                      
    Input parameters: List of ICOS station objects
                      (var_name: 'st_obj_ls', var_type: List)

    Output:           Python Widgets
    
    """

    #Create lookup dataframe:
    df_lookup = create_lookup_df_atc_L2(st_obj_ls)
    
    #Convert data type of date-columns from string to datetime:
    df_lookup.timeStart = [datetime.strptime(df_lookup.timeStart.iloc[i],'%Y-%m-%dT%H:%M:%SZ')
                    for i in range(len(df_lookup))]
    df_lookup.timeEnd = [datetime.strptime(df_lookup.timeEnd.iloc[i],'%Y-%m-%dT%H:%M:%SZ')
                  for i in range(len(df_lookup))]
    
    #Create widgets:
    tracer = Dropdown(options = create_prod_label(['CH4', 'CO', 'CO2'], sorted(df_lookup['specLabel'].unique().tolist())))
    station =  SelectMultiple(options = sorted(create_station_labels(df_lookup, 'stationName', 'stationId', 'samplingheight'),
                                         key = lambda x: (x[0].split(' (', 1)[0], float(x[1][1]))))
    
    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer and station/s:
    def update_plot_func(Tracer, Station, start_date, end_date):
        
        #Check user input:
        if(check_input_icos_L2_basic_stat(Station, start_date, end_date)):

            #Get a list of sublists, where every sublist contains the following:
            #1. ICOS Level-2 data object URL
            #2. ICOS Station ID (3-character code)
            #3. ICOS Station Sampling Height
            #4. Tracer/Gas (e.g. 'co2')
            #that refer to the selected station(s) and tracer:
            selection_dobj_url_list = [[df_lookup.dobj.loc[(df_lookup['stationId']==station[0]) &
                                                           (df_lookup['samplingheight']==station[1]) &
                                                           (df_lookup['specLabel']==Tracer[0]) &
                                                           (((df_lookup['timeStart']<=pd.Timestamp(start_date))&
                                                             (df_lookup['timeEnd']>=pd.Timestamp(start_date)))|
                                                            ((df_lookup['timeStart']<=pd.Timestamp(end_date))&
                                                             (df_lookup['timeEnd']>=pd.Timestamp(end_date)))|
                                                            ((df_lookup['timeStart']>=pd.Timestamp(start_date))&
                                                             (df_lookup['timeEnd']<=pd.Timestamp(end_date))))].values,
                                        station[0],
                                        station[1],
                                        Tracer[1].lower()]
                                       for station in Station]

            #Get a list of items, where every item represents the size of the url-array for every selected station:
            check_url_ls_size = [selection_dobj_url_list[m][0].size for m in range(len(selection_dobj_url_list))]

            #If Level-2 data are available for the selected tracer, station(s) and time period:
            if(sum(check_url_ls_size)>0):

                #Get a list of lists, where every sublist contains a data-object-ID, the station code, 
                #the sampling height & the selected tracer.
                #E.g. ['U4VYazHdmZwzr7DxUowMtUu-', 'co2', 'GAT', '30.0']:
                selection_list = [[selection_dobj_url_list[i][0][j],
                                   selection_dobj_url_list[i][3],
                                   selection_dobj_url_list[i][1],
                                   selection_dobj_url_list[i][2]]
                                  for i in range(len(selection_dobj_url_list))
                                  for j in range(len(selection_dobj_url_list[i][0]))]

                #Get a list of tuples (e.g. ('HPB', '50.0', 'co2')) with unique occurances of
                #"station code" - "station sampling height" - "tracer" triplets:
                station_unique_ls = list(set([(item[2], item[3], item[1]) for item in selection_list]))

                #Group selection_list items refering to tracer-data from the same station and sampling height to lists:  
                station_dobj_ls = [[item for item in selection_list
                                    if((item[1]==station_id[2]) & (item[2]==station_id[0]) & (item[3]==station_id[1]))]
                                   for station_id in station_unique_ls]
                
                #Call function to calculate and return basic statistics dataframe for the selected stations:
                return update_basic_statistics_binary(station_dobj_ls, start_date, end_date, st_obj_ls)
                

            #If no data are available for the selected tracer, station(s) and time period:
            else:
                print("\033[0;31;1m No ICOS Atmospheric Level-2 data available.\033[0;31;0m\n\n")



    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracer=tracer,
                                 Station=station,
                                 start_date=DatePicker(description='Starting Date',disabled=False),
                                 end_date=DatePicker(description='Ending Date',disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '430px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 2px'
    interact_c.widget.children[1].layout.width = '430px'
    interact_c.widget.children[1].layout.height = '115px'
    interact_c.widget.children[2].layout.width = '430px'
    interact_c.widget.children[3].layout.width = '430px'
    interact_c.widget.children[4].description = 'Update Table'
    interact_c.widget.children[4].button_style = 'danger'
    interact_c.widget.children[4].style.button_color = '#3973ac'
    interact_c.widget.children[4].layout.margin = '10px 10px 40px 180px' # top/right/bottom/left

In [ ]:
def create_widgets_correlation_multiple_tracers_multiple_stations(st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Wed Nov 18 18:40:00 2020
    Version:          0.1.3
    Author(s):        Karolina Pantazatou
    
    Description:      Function that creates a set of widgets; a station multiselect dropdown list, a
                      tracer multiselect dropdown list and a button. The function populates the dropdown
                      lists with values and outputs the result.
                      
                      
    Input parameters: List of ICOS station objects
                      (var_name: 'st_obj_ls', var_type: List)

    Output:           Python Widgets
    
    """

    #Create lookup dataframe:
    df_lookup = create_lookup_df_atc_L2(st_obj_ls)
    
    #Convert data type of date-columns from string to datetime:
    df_lookup.timeStart = [datetime.strptime(df_lookup.timeStart.iloc[i],'%Y-%m-%dT%H:%M:%SZ')
                    for i in range(len(df_lookup))]
    df_lookup.timeEnd = [datetime.strptime(df_lookup.timeEnd.iloc[i],'%Y-%m-%dT%H:%M:%SZ')
                  for i in range(len(df_lookup))]
    
    #Create widgets:
    tracers = SelectMultiple(options = create_prod_label(['CH4', 'CO', 'CO2'], sorted(df_lookup['specLabel'].unique().tolist())))
    stations =  SelectMultiple(options = sorted(create_station_labels(df_lookup, 'stationName', 'stationId', 'samplingheight'),
                                               key = lambda x: (x[0].split(' (', 1)[0], float(x[1][1]))))
    

    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Tracers, Stations, start_date, end_date):
        
        #Control input parameters: 
        if(check_input_icos_L2_correlation(Tracers, Stations, start_date, end_date)):
        
            #Get a list of sublists, where every sublist contains the following:
            #1. ICOS Level-2 data object URL
            #2. ICOS Station ID (3-character code)
            #3. ICOS Station Sampling Height
            #4. Tracer/Gas (e.g. 'co2')
            #that refer to the selected station(s) and tracer(s):
            selection_dobj_url_list = [[df_lookup.dobj.loc[(df_lookup['stationId']==station[0]) &
                                                           (df_lookup['samplingheight']==station[1]) &
                                                           (df_lookup['specLabel']==tracer[0]) &
                                                           (((df_lookup['timeStart']<=pd.Timestamp(start_date))&
                                                             (df_lookup['timeEnd']>=pd.Timestamp(start_date)))|
                                                            ((df_lookup['timeStart']<=pd.Timestamp(end_date))&
                                                             (df_lookup['timeEnd']>=pd.Timestamp(end_date)))|
                                                            ((df_lookup['timeStart']>=pd.Timestamp(start_date))&
                                                             (df_lookup['timeEnd']<=pd.Timestamp(end_date))))].values,
                                        tracer[1].lower(),
                                        station[0],
                                        station[1]]
                                       for tracer in Tracers for station in Stations]
            
            #Get a list of items, where every item represents the size of the url-array for every selected station:
            check_url_ls_size = [selection_dobj_url_list[m][0].size for m in range(len(selection_dobj_url_list))]

            #If Level-2 data are available for the selected tracer(s), station(s) and time period:
            if(sum(check_url_ls_size)>0):

                #Get a list of lists, where every sublist contains a data-object-ID, the station code, 
                #the sampling height & the selected tracer.
                #E.g. ['U4VYazHdmZwzr7DxUowMtUu-', 'GAT', '30.0', 'co2']:
                selection_list = [[selection_dobj_url_list[i][0][j],
                                   selection_dobj_url_list[i][1],
                                   selection_dobj_url_list[i][2],
                                   selection_dobj_url_list[i][3]]
                                  for i in range(len(selection_dobj_url_list))
                                  for j in range(len(selection_dobj_url_list[i][0]))]

                #Get a list of tuples (e.g. ('HPB', '50.0', 'co2')) with unique occurances of
                #"station code" - "station sampling height" - "tracer" triplets:
                station_unique_ls = list(set([(item[1], item[2], item[3]) for item in selection_list]))

                #Group selection_list items refering to tracer-data from the same station and sampling height to lists:  
                station_dobj_ls = [[item for item in selection_list
                                    if((item[1]==station_id[0]) & (item[2]==station_id[1]) & (item[3]==station_id[2]))]
                                   for station_id in station_unique_ls]


                #Call function to calculate and return a correlation statistics dataframe for the selected station:
                return update_corr_stat_multi_binary(station_dobj_ls, start_date, end_date, st_obj_ls)

            
            #If no data are available for the selected tracer, station(s) and time period:
            else:
                print("\033[0;31;1m No ICOS Atmospheric Level-2 data available.\033[0;31;0m\n\n")



    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracers = tracers,
                                 Stations = stations,
                                 start_date=DatePicker(description='Starting Date',disabled=False),
                                 end_date=DatePicker(description='Ending Date',disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '430px'
    interact_c.widget.children[0].layout.height = '60px'
    interact_c.widget.children[1].layout.width = '430px'
    interact_c.widget.children[2].layout.width = '430px'
    interact_c.widget.children[3].layout.width = '430px'
    interact_c.widget.children[4].description = 'Update Table'
    interact_c.widget.children[4].button_style = 'danger'
    interact_c.widget.children[4].style.button_color = '#3973ac'
    #interact_c.widget.children[4].layout.width = '300px'
    interact_c.widget.children[4].layout.margin = '10px 10px 20px 180px' # top/right/bottom/left
    

In [ ]:
def create_smoothing_widgets(st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Wed Nov 18 19:10:00 2020
    Version:          0.1.3
    Author(s):        Karolina Pantazatou
    
    Description:      Function that creates a set of widgets; a station dropdown list, a tracer dropdown
                      list, a 'total num of days' slider, a color-picker and a button. The function populates
                      the dropdown lists and slider with values and outputs the result.
                      
                      
    Input parameters: List of ICOS station objects
                      (var_name: 'st_obj_ls', var_type: List)

    Output:           Python Widgets
    
    """
    
    #Import modules:
    from bokeh.plotting import figure
    
    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

    #Create lookup dataframe:
    df_lookup = create_lookup_df_atc_L2(st_obj_ls)

    #Create widgets:
    tracer = Dropdown(options = create_prod_label(['CH4', 'CO', 'CO2'], sorted(df_lookup['specLabel'].unique().tolist())))
    station =  Dropdown(options = sorted(create_station_labels(df_lookup, 'stationName', 'stationId', 'samplingheight'),
                                         key = lambda x: (x[0].split(' (', 1)[0], float(x[1][1]))))
    
    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Tracer, Station, Days, Color, Citation):
        
        #Get a list of data obect URLs that refer to the selected station and tracer:
        data_obj_url_ls = list(df_lookup.dobj.loc[(df_lookup['stationId']==Station[0]) &
                                                  (df_lookup['samplingheight']==Station[1]) &
                                                  (df_lookup['specLabel']==Tracer[0])])
        
        #If L2-data is available for the selected tracer and station:
        if(len(data_obj_url_ls)>0):
            
            #Call function to return plot and map for the selected station:
            update_smoothing_plot_binary(data_obj_url_ls, Station, Tracer[1].lower(), Days, Color, st_obj_ls)
            
            
            #If the "citation" checkbox is checked:
            if(Citation):
                
                #Get a list with citation info for every ICOS Level-2 data object:
                cit_ls_L2 = [RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                             for dobj in data_obj_url_ls]
                    
                #Print citation title:
                print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                    
                #Loop through all citations:
                for cit in cit_ls_L2:
                    
                    #Print data object citation:
                    printmd("<sub>"+cit+"</sub>")
                    
        
        #If no L2-data are available for the selected tracer and station:
        else:
            print('\033[0;31;1m '+ 'No '+Tracer[1].upper().translate(SUB)+' Level-2 data available for the selected station' +'\033[0;31;0m\n\n')
        
    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracer=tracer,
                                 Station=station,
                                 Days = (0,90),
                                 Color=ColorPicker(concise=False,
                                                   description='Pick a color',
                                                   value='#3973ac',
                                                   disabled=False),
                                 Citation=Checkbox(value=True, description='Citation', disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '420px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 2px'
    interact_c.widget.children[1].layout.width = '420px'
    interact_c.widget.children[2].layout.width = '420px'
    interact_c.widget.children[3].layout.width = '420px'
    interact_c.widget.children[4].layout.width = '420px'
    interact_c.widget.children[5].description = 'Update Plot'
    interact_c.widget.children[5].button_style = 'danger'
    interact_c.widget.children[5].style.button_color = '#3973ac'
    interact_c.widget.children[5].layout.margin = '10px 10px 40px 180px' # top/right/bottom/left

In [ ]:
def create_widgets_comparing(st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Wed Nov 18 19:20:00 2020
    Version:          0.1.3
    Author(s):        Karolina Pantazatou
    
    Description:      Function that creates a set of widgets; a multiselect station dropdown list, a
                      tracer dropdown list and a button. The function populates the dropdown lists
                      with values and outputs the result.
                      
                      
    Input parameters: List of ICOS station objects
                      (var_name: 'st_obj_ls', var_type: List)

    Output:           Python Widgets
    
    """

    #Create lookup dataframe:
    df_lookup = create_lookup_df_atc_L2(st_obj_ls)

    #Create widgets:
    tracer = Dropdown(options = create_prod_label(['CH4', 'CO', 'CO2'], sorted(df_lookup['specLabel'].unique().tolist())))
    station =  SelectMultiple(options = sorted(create_station_labels(df_lookup, 'stationName', 'stationId', 'samplingheight'),
                                               key = lambda x: (x[0].split(' (', 1)[0], float(x[1][1]))))
    
    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Tracer, Station, Citation):
        
        #Get a list of sublists, where every sublist contains the following:
        #1. ICOS Level-2 data object URL
        #2. Tracer/Gas (e.g. 'co2')
        #3. ICOS Station ID (3-character code)
        #4. ICOS Station Sampling Height
        #that refer to the selected station(s) and tracer:
        selection_dobj_url_list = [[df_lookup.dobj.loc[(df_lookup['stationId']==station[0]) &
                                                       (df_lookup['samplingheight']==station[1]) &
                                                       (df_lookup['specLabel']==Tracer[0])].values,
                                    Tracer[1].lower(),
                                    station[0],
                                    station[1]]
                                   for station in Station
                                   if df_lookup.dobj.loc[(df_lookup['stationId']==station[0]) &
                                                         (df_lookup['samplingheight']==station[1]) &
                                                         (df_lookup['specLabel']==Tracer[0])].values.size>0]
        
        
        #If Level-2 data are available for the selected tracer and station(s):
        if(len(selection_dobj_url_list)>0):

            #Get a list of lists, where every sublist contains a data-object-ID, the station code, 
            #the sampling height & the selected tracer.
            #E.g. ['U4VYazHdmZwzr7DxUowMtUu-', 'co2', 'GAT', '30.0']:
            selection_list = [[selection_dobj_url_list[i][0][j],
                               selection_dobj_url_list[i][1],
                               selection_dobj_url_list[i][2],
                               selection_dobj_url_list[i][3]]
                              for i in range(len(selection_dobj_url_list))
                              for j in range(len(selection_dobj_url_list[i][0]))]
            
            ####
            #ICOS Atmospheric Level-2 Data for a given station, a given tracer and  
            #a given sampling height can be stored in two different files in cases
            #where the measuring instrument has changed.
            #The following code controls for such occurances and merges the data
            #if necessary.
            ####
            
            #Get a list of tuples (e.g. ('HPB', '50.0', 'co2')) with unique occurances of
            #"station code" - "station sampling height" - "tracer" triplets:
            station_unique_ls = list(set([(item[1], item[2], item[3]) for item in selection_list]))
            
            #Group selection_list items refering to tracer-data from the same station and sampling height to lists:  
            station_dobj_ls = [[item for item in selection_list
                                if((item[1]==station_id[0]) & (item[2]==station_id[1]) & (item[3]==station_id[2]))]
                               for station_id in station_unique_ls]
            
            #Get plot displaying tracer-values for the selected station/s:
            update_comparing_binary(station_dobj_ls, len(station_unique_ls), st_obj_ls)
            
            #If the "citation" checkbox is checked:
            if(Citation):
                
                #Get a list with citation info for every ICOS Level-2 data object:
                cit_ls_L2 = [RunSparql(sparql_query=sparqls.get_icos_citation(dobj[0][0]), output_format='pandas').run().cit.iloc[0]
                             for dobj in selection_dobj_url_list]
                    
                #Print citation title:
                print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                    
                #Loop through all citations:
                for cit in cit_ls_L2:
                    
                    #Print data object citation:
                    printmd("<sub>"+cit+"</sub>")
        
        #If no L2-data is available for the selected tracer and station:
        else:
            print('\033[0;31;1m '+ 'No Level-2 data available for the selected tracer and/or station/s.' +'\033[0;31;0m\n\n')
            

            

    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracer=tracer,
                                 Station=station,
                                 Citation=Checkbox(value=True, description='Citation', disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '430px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 2px'
    interact_c.widget.children[1].layout.width = '430px'
    interact_c.widget.children[1].layout.height = '115px'
    interact_c.widget.children[2].layout.width = '430px'
    interact_c.widget.children[3].description = 'Update Plot/s'
    interact_c.widget.children[3].button_style = 'danger'
    interact_c.widget.children[3].style.button_color = '#3973ac'
    interact_c.widget.children[3].layout.margin = '10px 10px 40px 180px' # top/right/bottom/left

In [ ]:
def create_widgets_L1(st_obj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Wed Nov 18 19:30:00 2020
    Version:          0.1.3
    Author(s):        Karolina Pantazatou
    
    Description:      Function that creates a set of widgets; a station dropdown list, a tracer
                      dropdown list, a color-picker, a checkbox and a button. The function populates
                      the dropdown lists with values and outputs the result.
                      
                      
    Input parameters: List of ICOS station objects
                      (var_name: 'st_obj_ls', var_type: List)

    Output:           Python Widgets
    
    """
    
    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    
    #Create lookup dataframes:
    df_lookup_L1 = create_lookup_df_atc_L1(st_obj_ls) #ICOS Level 1 Atmosphere Data
    df_lookup_L2 = create_lookup_df_atc_L2(st_obj_ls) #ICOS Level 2 Atmosphere Data


    #Create widgets:
    tracer_L1 = Dropdown(options = create_prod_label(['CH4', 'CO', 'CO2'], sorted(df_lookup_L1['specLabel'].unique().tolist())))
    station_L1 =  Dropdown(options = sorted(create_station_labels(df_lookup_L1, 'stationName', 'stationId', 'samplingheight'),
                                            key = lambda x: (x[0].split(' (', 1)[0], float(x[1][1]))))
    

    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Tracer, Station, color, Level, Citation):
        
        #If "Add Level 2 Data" is not selected and if L1-data is available for the specific station and tracer:
        if(Level==False):
            
            #Get a list of L1 data obect URLs that refer to the selected station and tracer:
            data_obj_url_L1_ls = list(df_lookup_L1.dobj.loc[(df_lookup_L1['stationId']==Station[0]) &
                                                            (df_lookup_L1['samplingheight']==Station[1]) &
                                                            (df_lookup_L1['specLabel']==Tracer[0])])
            
            #If L1-data is available for the selected tracer and station:
            if(len(data_obj_url_L1_ls)>0):
                
                #Call function to return plot for the selected station (Level 1 Data):
                update_icos_single_station_plot_LX_binary(data_obj_url_L1_ls, Station[0], Station[1], Tracer[1].lower(), color, 1, st_obj_ls)
                
                #If the "citation" checkbox is checked:
                if(Citation):

                    #Get a list with citation info for every ICOS Level-1 data object:
                    cit_ls_L1 = [RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                 for dobj in data_obj_url_L1_ls]

                    #Print citation title:
                    print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')

                    #Loop through all citations:
                    for cit in cit_ls_L1:

                        #Print data object citation:
                        printmd("<sub>"+cit+"</sub>")
            
            
            #If no L1-data is available for the selected tracer and station:
            else:
                print('\033[0;31;1m '+ 'No '+Tracer[1].upper().translate(SUB)+' Level-1 data available for the selected station' +'\033[0;31;0m\n\n')
                
           
        
        #If "Add Level 2 Data" is selected:
        elif(Level==True):
            
            #Get a list of L1 data object URLs that refer to the selected station and tracer:
            data_obj_url_L1_ls = list(df_lookup_L1.dobj.loc[(df_lookup_L1['stationId']==Station[0]) &
                                                            (df_lookup_L1['samplingheight']==Station[1]) &
                                                            (df_lookup_L1['specLabel']==Tracer[0])])
            
            #Get a list of L2 data object URLs that refer to the selected station and tracer:
            data_obj_url_L2_ls = list(df_lookup_L2.dobj.loc[(df_lookup_L2['stationId']==Station[0]) &
                                                            (df_lookup_L2['samplingheight']==Station[1]) &
                                                            (df_lookup_L2['specLabel']=='ICOS ATC '+Tracer[1]+' Release')])
            
            #If Level-1 & Level-2 data are available for the selected tracer and station:
            if((len(data_obj_url_L1_ls)>0) & (len(data_obj_url_L2_ls)>0)):
                
                #Call function to return plot for the selected station (Level 1 & 2 Data):
                update_icos_single_station_plot_L1_L2_binary(data_obj_url_L1_ls, data_obj_url_L2_ls, Station[0], Station[1], Tracer[1].lower(), color, st_obj_ls)
                
                #If the "citation" checkbox is checked:
                if(Citation):
                    
                    #Get a list with citation info for every ICOS Level-1 data object:
                    cit_ls_L1 = [RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                 for dobj in data_obj_url_L1_ls]

                    #Get a list with citation info for every ICOS Level-2 data object:
                    cit_ls_L2 = [RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                 for dobj in data_obj_url_L2_ls]

                    #Concatenate citation lists to one list: 
                    cit_ls = cit_ls_L1 + cit_ls_L2

                    #Print citation:
                    print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')

                    #Loop through all citations:
                    for cit in cit_ls:
                        
                        #Print data object citation:
                        printmd("<sub>"+cit+"</sub>")
            
            
            #If only Level-1 data are available for the selected tracer and station:
            elif((len(data_obj_url_L1_ls)>0) & (len(data_obj_url_L2_ls)<1)):
                
                #Print message:
                print('\033[0;31;1m '+ 'No Level-2 data available yet ...' +'\033[0;31;0m\n\n')                
                
                #Call function to return plot for the selected station (Level 1):
                update_icos_single_station_plot_LX_binary(data_obj_url_L1_ls, Station[0], Station[1], Tracer[1].lower(), color, 1, st_obj_ls)
                
                #If the "citation" checkbox is checked:
                if(Citation):
                    
                    #Get a list with citation info for every ICOS Level-1 data object:
                    cit_ls_L1 = [RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                 for dobj in data_obj_url_L1_ls]


                    #Print citation:
                    print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')

                    #Loop through all citations:
                    for cit in cit_ls_L1:
                        
                        #Print data object citation:
                        printmd("<sub>"+cit+"</sub>")
            
            
            #If only Level-2 data are available for the selected tracer and station:
            elif((len(data_obj_url_L1_ls)<1) & (len(data_obj_url_L2_ls)>0)):
                
                #Print message:
                print('\033[0;31;1m '+ 'No Level-1 data available ...' +'\033[0;31;0m\n\n')
                
                #Call function to return plot for the selected station (Level-2 Data):
                update_icos_single_station_plot_LX_binary(data_obj_url_L2_ls, Station[0], Station[1], Tracer[1].lower(), color, 2, st_obj_ls)
                
                #If the "citation" checkbox is checked:
                if(Citation):
                    
                    #Get a list with citation info for every ICOS Level-1 data object:
                    cit_ls_L2 = [RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                 for dobj in data_obj_url_L2_ls]


                    #Print citation:
                    print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')

                    #Loop through all citations:
                    for cit in cit_ls_L2:
                        
                        #Print data object citation:
                        printmd("<sub>"+cit+"</sub>")
            
            
            #If no L1-data or L2-data are available for the selected tracer and station:
            else:
                print('\033[0;31;1m '+ 'No Level-1 or Level-2 data available for the selected tracer and station at present.\nTry a new combination!' +'\033[0;31;0m\n\n')


    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracer=tracer_L1,
                                 Station=station_L1,
                                 color=ColorPicker(concise=False,description='Pick a color',value='#3973ac',
                                                   disabled=False),
                                 Level=Checkbox(value=False, description='Add Level 2 Data', disabled=False),
                                 Citation=Checkbox(value=True, description='Citation', disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '430px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 2px'
    interact_c.widget.children[1].layout.width = '430px'
    interact_c.widget.children[2].layout.width = '430px'
    interact_c.widget.children[3].layout.width = '430px'
    interact_c.widget.children[3].layout.margin = '12px 2px 2px 2px'
    interact_c.widget.children[4].layout.width = '430px'
    interact_c.widget.children[5].description = 'Update Plot'
    interact_c.widget.children[5].button_style = 'danger'
    interact_c.widget.children[5].style.button_color = '#3973ac'
    interact_c.widget.children[5].layout.margin = '10px 10px 40px 180px' # top/right/bottom/left

<a id='control_input'></a>
<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>
<br>
<br>

## 9. Control input
This part includes functions that control the input to functions with statistics.

In [ ]:
def check_input_icos_L2_basic_stat(Station, start_date, end_date):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri May 17 14:30:00 2019
    Last Changed:     Fri May 17 14:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that checks the user input to widgets and returns a boolean for
                      invalid or empty input.
                      
    Input parameters: 1. List of sublists, where every sublist contains the Station ID & Sampling Height
                         (var_name: 'Station', var_type: List)
                      2. Start date of time period
                         (var_name: 'start_date', var_type: DateTime Object)
                      3. End date of time period
                         (var_name: 'end_date', var_type: DateTime Object)

    Output:           Pandas DataFrame
    
    """
    
    #Check if NO station has been selected:
    if(len(Station)<1):

        #Print message:
        print(("\033[0;31;1m Select station! \033[0;31;0m\n\n"))

        #Return boolean:
        return False
        
    #Check if a station has been selected:
    else:

        #Check if a start-date and/or end-date have been selected:
        if((start_date==None)|(end_date==None)):

            #Print message:
            print(("\033[0;31;1m Select a start date and/or end date! \033[0;31;0m\n\n"))

            #Return boolean:
            return False

        #If a start-date and an end-date have been selected:
        else:

            #Compute the difference between end_date and start_date:
            diff = end_date - start_date

            #Check if end-date refers to an earlier date than start-date:
            if(diff.days<0):

                #Print message:
                print('\033[0;31;1m Error...\n The selected start-date corresponds to a later date than the selected end-date.\n Enter new dates!\n\n')

                #Return boolean:
                return False
            
            #If the selected start-date & end-date are valid:
            else:
                
                #Return boolean:
                return True

  

In [ ]:
def check_input_icos_L2_correlation(Tracer, Station, start_date, end_date):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri May 17 14:30:00 2019
    Last Changed:     Fri May 17 14:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that checks the user input to widgets and returns a boolean for
                      invalid or empty input.
                      
    Input parameters: 1. List with Tracer(s)/gas(es) (Long Text)
                         (var_name: 'Tracer', var_type: List)
                      2. List of sublists, where every sublist contains the Station ID & Sampling Height
                         (var_name: 'Station', var_type: List)
                      3. Start date of time period
                         (var_name: 'start_date', var_type: DateTime Object)
                      4. End date of time period
                         (var_name: 'end_date', var_type: DateTime Object)

    Output:           Pandas DataFrame
    
    """
    
    #Check if NO tracer has been selected:
    if(len(Tracer)<1):

        #Print message:
        print(("\033[0;31;1m Select tracer! \033[0;31;0m\n\n"))

        #Return boolean:
        return False
        
    #Check if a tracer has been selected:
    else:
    
        #Check if NO station has been selected:
        if(len(Station)<1):

            #Print message:
            print(("\033[0;31;1m Select station! \033[0;31;0m\n\n"))

            #Return boolean:
            return False

        #Check if a station has been selected:
        else:

            #Check if a start-date and/or end-date have been selected:
            if((start_date==None)|(end_date==None)):

                #Print message:
                print(("\033[0;31;1m Select a start date and/or end date! \033[0;31;0m\n\n"))

                #Return boolean:
                return False

            #If a start-date and an end-date have been selected:
            else:

                #Compute the difference between end_date and start_date:
                diff = end_date - start_date

                #Check if end-date refers to an earlier date than start-date:
                if(diff.days<0):

                    #Print message:
                    print('\033[0;31;1m Error...\n The selected start-date corresponds to a later date than the selected end-date.\n Enter new dates!\n\n')

                    #Return boolean:
                    return False

                #If the selected start-date & end-date are valid:
                else:

                    #Return boolean:
                    return True

  

<a id='statistics'></a>
<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>
<br>
<br>

## 10.Statistics
This part includes functions that calculate statistics for ICOS Level 1 and Level 2 CO, CO$_2$ and CH$_4$ data products.

In [ ]:
def calculate_basic_statistics_binary(station_df_ls, tracer):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Wed May 15 14:30:00 2019
    Last Changed:     Wed May 15 14:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that loops through a list of sublists, where every sublist contains
                      the data-dataframe. dictionary with station info and dictionary with tracer info 
                      of a specific station, and computes the min,
                      max, mean and standard deviation of the tracer-column in the data-dataframe
                      of every station. The statistical values are stored in a separate dataframe.
                      One dataframe is produced for every station. In cases where more than one
                      stations have been selected, all separate dataframes with basic statistics
                      results per station are concatenated to one dataframe that is then returned
                      as output.
                      
    Input parameters: 1. List with sublists of data-dataframes for every selected station
                         (var_name: 'station_df_ls', var_type: List)
                      2. Tracer/gas, e.g. 'co2'
                         (var_name: 'tracer', var_type: String)

    Output:           Pandas DataFrame
    
                      Columns:
                      1. The earliest date in the dataset
                         (var_name: "start_date", var_type: NumPy DateTime64)
                      2. The latest date in the dataset
                         (var_name: "end_date", var_type: NumPy DateTime64)
                      3. The minimum tracer value in the dataset
                         (var_name: 'min', var_type: float)
                      4. The maximum tracer value in the dataset
                         (var_name: 'max', var_type: float)
                      5. The average tracer value in the dataset
                         (var_name: 'mean', var_type: float)
                      6. The standard deviation of the tracer values in the dataset
                         (var_name: 'st_dev', var_type: float)
    
    """
    

    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

    #Create list to store the metadata and data dataframes for every station:
    df_list = []
    
    #Loop through every station's metadata- & data- dataframe: 
    for df in station_df_ls:
        
        #Get time-period and statistics:
        stat_df = pd.DataFrame({'start_date': df[0].index.values.min(),
                                'end_date': df[0].index.values.max(),
                                'min': round(pd.to_numeric(df[0][tracer],errors='coerce').min(),2),
                                'max': round(pd.to_numeric(df[0][tracer],errors='coerce').max(),2),
                                'mean': round(pd.to_numeric(df[0][tracer],errors='coerce').mean(),2),
                                'st dev': round(pd.to_numeric(df[0][tracer],errors='coerce').std(),2)}, 
                               index=[df[1]['station_name']+', '+
                                      df[1]['station_sampling_height']+
                                      ' ('+tracer.upper().translate(SUB)+')'])
        
        #Add dataframe to list:
        df_list.append(stat_df)              
            
        
    #Concatenate dataframes to one dataframe:
    basic_statistics_df = pd.concat(df_list)
    
    #Sort dataframe index:
    basic_statistics_df.sort_index(inplace=True)

    #Return dataframe:
    return basic_statistics_df


<br>
<br>
<br>
<br>
<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>